In [ ]:
import curvature as c
import manifold as m
import numpy as np

Sampling from each of the manifolds: examples below. Added two functions to each class (exact_curvatures and mse) that calculate the exact curvatures (given in a list called Tscs for True Scalar Curvatures) and the mean squared error.

In [ ]:
X1 = m.Sphere().sample(1000, 2)
Tscs1 = m.Sphere().exact_curvatures(X1)

X2 = m.Euclidean().sample(1000, 2, 1)
Tscs2 = m.Euclidean().exact_curvatures(X2)

# to-do: format torus like all other manifold classes? get thetas from X values so we don't have to unpack
X3, thetas = m.Torus().sample(1000, 0.5, 1)
Tscs3 = m.Torus().exact_curvatures(thetas, 0.5, 1)

X4 = m.PoincareDisk().sample(1000)
Tscs4 = m.PoincareDisk().exact_curvatures(X4)

X5 = m.Hyperboloid().sample(1000, 2, 1, 2)
Tscs5 = m.Hyperboloid().exact_curvatures(X5)

In [ ]:
import math

sce1 = c.scalar_curvature_est(X1, 2)
Ss1 = sce1.estimate_all(rmax = 2) # list of length N, where N = number of points in X. Ss[i] = scalar curvature estimate at point i in X

sce2 = c.scalar_curvature_est(X2, 2)
Ss2 = sce2.estimate_all(rmax = 1)

sce3 = c.scalar_curvature_est(X3, 2)
Ss3 = sce3.estimate_all(rmax = 1)

sce4 = c.scalar_curvature_est(X4, 2)
Ss4 = sce4.estimate_all(rmax = 1)

sce5 = c.scalar_curvature_est(X5, 2)
Ss5 = sce5.estimate_all(rmax = 1)

In [ ]:
mse1 = m.Sphere().mse(X1, Ss1)
print(mse1)

mse2 = m.Euclidean().mse(X2, Ss2)
print(mse2)

mse3 = m.Torus().mse(thetas, 0.5, 1, Ss3)
print(mse3)

mse4 = m.PoincareDisk().mse(X4, Ss4)
print(mse4)

mse5 = m.Hyperboloid().mse(X5, Ss5)
print(mse5)

6/17: Testing New Functions

c.scalar_curvature_est

    rmax_from_nballs

    dist_to_nn

    rs_from_spacing

    fit_quad_coeff

    estimate_all
    
Ok as far as I know, I haven't broken anything! Yay :D

In [ ]:
dists = np.sort(sce1.get_Rdist()[0, :])
print(dists)

rmax = sce1.rmax_from_nballs(0, 10)
print(rmax)

nearest = sce1.dist_to_nn(0)
print(nearest)

rs = sce1.rs_from_spacing(0, 10, 0.1)
print(rs)

In [ ]:
Xtest = m.Sphere().sample(1000, 2)
Tscs = m.Sphere().exact_curvatures(Xtest)
scetest = c.scalar_curvature_est(Xtest, 2)
print(type(scetest))

SsA = scetest.estimate_all(rmax = 2)
# mseA = m.Sphere.mse(Xtest, SsA)

SsB = scetest.estimate_all(rs = [0.1, 0.2, 0.3, 0.4, 0.5])
# mseB = m.Sphere.mse(Xtest, SsB)

SsC = scetest.estimate_all(nballs = 10)
# mseC = m.Sphere.mse(Xtest, SsC)

SsD = scetest.estimate_all(nballs = 10, spacing = 0.1)
# mseD = m.Sphere.mse(Xtest, SsD)

SsE = scetest.estimate_all(nballs = 20, spacing = 0.1)
# mseE = m.Sphere.mse(Xtest, SsE)

SsF = scetest.estimate_all(nballs = 30, spacing = 0.1)
# mseF = m.Sphere.mse(Xtest, SsF)
# print(mseA, mseB, mseC, mseD, mseE, mseF)

estimates = [SsA, SsB, SsC, SsD, SsE, SsF]
for item in estimates:
    print(m.Sphere().mse(Xtest, item))



In [ ]:
obj = m.Sphere()
print(type(obj))

objTest = obj.sample(100, 2)


Values previously used in experiments:
* Euclidean: 10,000 points sampled uniformly from a disk of radius 2
* Sphere: 10,000 points sampled uniformly from S2
    * r_min = 0, r_max = pi/2
* Hyperbolic: 10,000 points sampled uniformly from a disk of radius 2 (is this a Poincare disk?)

estimate curvature on a subset of points (disk of radius 1) to avoid boundary effects

In [ ]:
import numpy as np

# edit: added radius as a parameter
def run_estimator(N, manifold, distType, kfunc, nn, fitVersion, nballs, spacing, radius, rmax = None):
    '''
    returns the mean square error for one experiment, given the parameters as inputs
    manifold: type of surface from which points are sampled
        0 = Sphere
        1 = Euclidean
        2 = Poincare Disk
        3 = Torus
        4 = Hyperboloid
    distType: type of distance metric used to get Rdist matrix
        0 = Estimated Riemannian distances (Isomap)
        1 = True Riemannian distances
        2 = Euclidean distances
    kfunc: type of kernel function used in density calculations
        0 = gauss
        1 = biweight
        2 = epanechnikov
        3 = triweight
    nn: number of nearest neighbors used in Isomap
    fitVersion: version of analytic solution for C used in quadratic fit function
    nballs: number of geodesic balls used in getting ball ratio sequence
    spacing: spacing between radii of adjacent balls if using uniform separation
    '''
    # for actual results, change sample size to 10000
    Tscs = None
    if manifold == 0:
        surf = m.Sphere()
        X = surf.sample(N, 2, R = radius)
    elif manifold == 1:
        surf = m.Euclidean()
        X = surf.sample(N, 2, radius)
    elif manifold == 2:
        surf = m.PoincareDisk()
        X = surf.sample(N, -1, radius)

    # Temporarily ommitting other manifolds because they don't have Rdist_array and distance_array functions
    '''
    elif manifold == 3:
        surf = m.Torus()
        X, thetas = surf.sample(N, 0.5, 1.5)
        Tscs = surf.exact_curvatures(thetas, 0.5, 1.5)
    else:
        surf = m.Hyperboloid()
        X = m.Hyperboloid.sample(N, 2, 1, 2)
    '''
    
    # compute true scalar curvature values at all sample points
    if Tscs == None:
        Tscs = surf.exact_curvatures(X)

    # calculates distance arrays in different ways
    if distType == 0:
        Rdist = None
    elif distType == 1:
        Rdist = surf.Rdist_array(X)
    else:
        Rdist = surf.distance_array(X)

    # creates estimator with nn and kfunc parameters, and estimates curvature at all points
    sce = c.scalar_curvature_est(X, 2, nn, kfunc, Rdist = Rdist, verbose = False)
    Ss = sce.estimate_all(rmax = rmax, nballs = nballs, spacing = spacing, version = fitVersion)

    # calculates and returns mse
    mse = surf.mse(X, Ss)

    return mse

kernels = {0: 'gauss', 1: 'biweight', 2: 'epanechnikov', 3: 'triweight'}


        



In [ ]:
# testing dictionary functionality, not a part of the function
'''
mses = {}

mse = run_estimator(0, 0, 0, 10, 1, 5, 0.1)
print("condition: ", manifold, distType, kernels[kfunc], nn, fitVersion, nballs, spacing)
print("mse: ", mse)
mses[(manifold, distType, kfunc, nn, fitVersion, nballs, spacing)] = mse

print(mses)
                        
for manifold in range(3):
    for distType in range(3):
        for kfunc in range(4):
            for nn in range (5, 30, 5):
                for fitVersion in range(1, 3, 1):
                    for nballs in range(5, 30, 5):
                        for spacing in [None, 0.1, 0.2, 0.3, 0.4, 0.5]:
                            mse = run_estimator(manifold, distType, kfunc, nn, fitVersion, nballs, spacing)
                            # print("condition: ", manifold, distType, kernels[kfunc], nn, fitVersion, nballs, spacing)
                            # print("mse: ", mse)
                            mses[(manifold, distType, kfunc, nn, fitVersion, nballs, spacing)] = mse
'''

In [ ]:
# Now want to make the above into a function to make life easier
def populate_mses(N, manifolds, distTypes, kfuncs, nnmax, fits, nballsmax, radius):
    
    mses = {}

    for manifold in range(manifolds):
        for distType in range(distTypes):
            for kfunc in range(kfuncs):
                for nn in range (20, nnmax + 1, 5):
                    for fitVersion in range(1, fits + 1, 1):
                        # currently manually changing this, pass in nballs as a param
                        for nballs in [25, 50, 75, 100]:
                            # currently manually changing this, pass in spacings as a param
                            for spacing in [None, 0.025, 0.05, 0.075, 0.1]:
                                mse = run_estimator(N, manifold, distType, kfunc, nn, fitVersion, nballs, spacing)
                                # print("condition: ", manifold, distType, kernels[kfunc], nn, fitVersion, nballs, spacing)
                                # print("mse: ", mse)
                                mses[(manifold, distType, kfunc, nn, fitVersion, nballs, spacing)] = mse
                        print('finished fit ', fitVersion, 'for ', nn, ' neighbors with kernel function ', kfunc, ' and distType ', distType)
    return mses

def avgmse_condition(manifold = -1, distType = -1, kfunc = -1, nn = -1, fitVersion = -1, nballs = -1, spacing = -1, mses = {}):
    conditions = [manifold, distType, kfunc, nn, fitVersion, nballs, spacing]
    match_errors = []
    for key in mses.keys():
        match = True
        for j, setting in enumerate(conditions):
            if (setting != -1) and (key[j] != setting):
                match = False
        if match:
            match_errors.append(mses[key])
    return np.mean(np.array(match_errors))

6/23/22: Some notes on some tests
1. Sample size: I ran the code in the cell below for 100, 1000, and 10000 sample points each to get a gauge for runtime locally.
    * 100 sample points per test: 54.7 seconds
        * gaussian vs. biweight = 7390.16 vs. 1664.66, 106791.46 vs. 2471.30
    * 1000 sample points per test: 173.3 seconds
        * gaussian vs. biweight = 3631813.44 vs. 2351835 lmao
    * 10000 sample poitns per test: 
        * too few neighbors for Isomap to have one connected component, also slowed down stuff significantly
        * Stopped after 500 seconds

NOTE: I messed with number of nearest neighbors, starts at 15 now so the cell below shouldn't work
Aahhh it's a little too many numbers to keep track of

Single parameter variance
Default: 10000 sample points, 1 manifold (S2), 3 distance types, 4 kernel functions, 15-25 neighbors, 2 fit versions

In [ ]:
# Conditions: only S2, isomap with gaussian / biweight kernels using 10 neighbors, only version 1 of the analytic fit, and 5 / 10 balls for regression
mse0 = populate_mses(1000, 1, 1, 2, 10, 1, 10)

gauss = avgmse_condition(-1, -1, 0, -1, -1, -1, -1, mse0)
biweight = avgmse_condition(-1, -1, 1, -1, -1, -1, -1, mse0)

print("average error with a gaussian kernel: ", gauss)
print("average error with a biweight kernel: ", biweight)


In [ ]:
print(mse0)

In [ ]:
mse1 = populate_mses(1000, 3, 3, 4, 20, 2, 25)

In [ ]:
print(mse1)

In [ ]:
print(mse0)

Results from MSE0 below (collapsed for space reasons)

{(0, 0, 0, 10, 1, 5, None): 41008623.71790418, (0, 0, 0, 10, 1, 5, 0.005): 47720574784.366005, (0, 0, 0, 10, 1, 5, 0.01): 787635260.2399367, (0, 0, 0, 10, 1, 5, 0.015): 83745533.76974644, (0, 0, 0, 10, 1, 5, 0.02): 12029988.320475183, (0, 0, 0, 10, 1, 5, 0.025): 3408587.301622031, (0, 0, 0, 10, 1, 10, None): 671551.7612566536, (0, 0, 0, 10, 1, 10, 0.005): 359448308.3137354, (0, 0, 0, 10, 1, 10, 0.01): 6815464.487448729, (0, 0, 0, 10, 1, 10, 0.015): 620779.1489648985, (0, 0, 0, 10, 1, 10, 0.02): 89708.11662720928, (0, 0, 0, 10, 1, 10, 0.025): 13705.007959886903, (0, 0, 1, 10, 1, 5, None): 19194985.723258477, (0, 0, 1, 10, 1, 5, 0.005): 38008717542.205986, (0, 0, 1, 10, 1, 5, 0.01): 545715111.8528018, (0, 0, 1, 10, 1, 5, 0.015): 42027325.35160712, (0, 0, 1, 10, 1, 5, 0.02): 8037258.356983489, (0, 0, 1, 10, 1, 5, 0.025): 1526988.1409536712, (0, 0, 1, 10, 1, 10, None): 302112.6374990651, (0, 0, 1, 10, 1, 10, 0.005): 572204684.1821575, (0, 0, 1, 10, 1, 10, 0.01): 3829885.588766015, (0, 0, 1, 10, 1, 10, 0.015): 215487.36078347667, (0, 0, 1, 10, 1, 10, 0.02): 22430.821997674168, (0, 0, 1, 10, 1, 10, 0.025): 2159.0091668740633}



Results from MSE1 below (collapsed for space reasons)

Results from MSE1:

{(0, 0, 0, 15, 1, 25, None): 19722.500620836407, (0, 0, 0, 15, 1, 25, 0.005): 1104874.8665898906, (0, 0, 0, 15, 1, 25, 0.01): 14973.710654044438, (0, 0, 0, 15, 1, 25, 0.015): 697.2250697680801, (0, 0, 0, 15, 1, 25, 0.02): 118.38394627465614, (0, 0, 0, 15, 1, 25, 0.025): 29.47614986861102, (0, 0, 0, 15, 2, 25, None): 29494.67614694566, (0, 0, 0, 15, 2, 25, 0.005): 1663565.9962221105, (0, 0, 0, 15, 2, 25, 0.01): 20483.723101429074, (0, 0, 0, 15, 2, 25, 0.015): 1063.4808067288004, (0, 0, 0, 15, 2, 25, 0.02): 133.22510240490698, (0, 0, 0, 15, 2, 25, 0.025): 36.21690770211453, (0, 0, 0, 20, 1, 25, None): 11707.50657442392, (0, 0, 0, 20, 1, 25, 0.005): 1461600.4338765582, (0, 0, 0, 20, 1, 25, 0.01): 15453.133284332456, (0, 0, 0, 20, 1, 25, 0.015): 803.5430023045135, (0, 0, 0, 20, 1, 25, 0.02): 93.75882315353195, (0, 0, 0, 20, 1, 25, 0.025): 14.735810661287267, (0, 0, 0, 20, 2, 25, None): 24094.56397213122, (0, 0, 0, 20, 2, 25, 0.005): 1803190.898008816, (0, 0, 0, 20, 2, 25, 0.01): 18045.78592381838, (0, 0, 0, 20, 2, 25, 0.015): 1155.525032041421, (0, 0, 0, 20, 2, 25, 0.02): 106.10918613273029, (0, 0, 0, 20, 2, 25, 0.025): 17.724203195607757, (0, 0, 1, 15, 1, 25, None): 1385.6595744028045, (0, 0, 1, 15, 1, 25, 0.005): 559366.4932285551, (0, 0, 1, 15, 1, 25, 0.01): 2323.8616464165075, (0, 0, 1, 15, 1, 25, 0.015): 290.50169852940166, (0, 0, 1, 15, 1, 25, 0.02): 218.54247412462206, (0, 0, 1, 15, 1, 25, 0.025): 118.4465480238652, (0, 0, 1, 15, 2, 25, None): 1556.689809394895, (0, 0, 1, 15, 2, 25, 0.005): 905304.798840152, (0, 0, 1, 15, 2, 25, 0.01): 2709.4877282526836, (0, 0, 1, 15, 2, 25, 0.015): 220.72415370376902, (0, 0, 1, 15, 2, 25, 0.02): 221.09346856503544, (0, 0, 1, 15, 2, 25, 0.025): 119.6160810723852, (0, 0, 1, 20, 1, 25, None): 1916.2358507300196, (0, 0, 1, 20, 1, 25, 0.005): 660376.1590567377, (0, 0, 1, 20, 1, 25, 0.01): 2307.5462572139327, (0, 0, 1, 20, 1, 25, 0.015): 109.4981548924484, (0, 0, 1, 20, 1, 25, 0.02): 102.82582581404641, (0, 0, 1, 20, 1, 25, 0.025): 94.32362978043277, (0, 0, 1, 20, 2, 25, None): 1727.2813419846027, (0, 0, 1, 20, 2, 25, 0.005): 970623.8048514285, (0, 0, 1, 20, 2, 25, 0.01): 3397.6249466271292, (0, 0, 1, 20, 2, 25, 0.015): 163.36098865439027, (0, 0, 1, 20, 2, 25, 0.02): 194.01866613273893, (0, 0, 1, 20, 2, 25, 0.025): 121.63865869298523, (0, 0, 2, 15, 1, 25, None): 1550.1444843660367, (0, 0, 2, 15, 1, 25, 0.005): 1224985.7103787197, (0, 0, 2, 15, 1, 25, 0.01): 4191.889172008995, (0, 0, 2, 15, 1, 25, 0.015): 172.89987097076565, (0, 0, 2, 15, 1, 25, 0.02): 127.58623186528689, (0, 0, 2, 15, 1, 25, 0.025): 100.91528318995884, (0, 0, 2, 15, 2, 25, None): 3097.025432529581, (0, 0, 2, 15, 2, 25, 0.005): 1080746.7950442259, (0, 0, 2, 15, 2, 25, 0.01): 5159.7324964572945, (0, 0, 2, 15, 2, 25, 0.015): 183.31759838849777, (0, 0, 2, 15, 2, 25, 0.02): 139.86356819519656, (0, 0, 2, 15, 2, 25, 0.025): 148.86906920278102, (0, 0, 2, 20, 1, 25, None): 2403.533514232248, (0, 0, 2, 20, 1, 25, 0.005): 960997.3380809057, (0, 0, 2, 20, 1, 25, 0.01): 3621.3614449973907, (0, 0, 2, 20, 1, 25, 0.015): 115.7066723196826, (0, 0, 2, 20, 1, 25, 0.02): 78.63412868964294, (0, 0, 2, 20, 1, 25, 0.025): 63.592365020075874, (0, 0, 2, 20, 2, 25, None): 3291.2648433021277, (0, 0, 2, 20, 2, 25, 0.005): 1183314.9156705826, (0, 0, 2, 20, 2, 25, 0.01): 4852.8280852687285, (0, 0, 2, 20, 2, 25, 0.015): 101.98650430739782, (0, 0, 2, 20, 2, 25, 0.02): 92.36109616803398, (0, 0, 2, 20, 2, 25, 0.025): 80.16099528132996, (0, 0, 3, 15, 1, 25, None): 1633.6873273440976, (0, 0, 3, 15, 1, 25, 0.005): 436360.47778589436, (0, 0, 3, 15, 1, 25, 0.01): 1658.849938884594, (0, 0, 3, 15, 1, 25, 0.015): 248.90060993921182, (0, 0, 3, 15, 1, 25, 0.02): 256.0678748695889, (0, 0, 3, 15, 1, 25, 0.025): 121.12148186043194, (0, 0, 3, 15, 2, 25, None): 1708.3172325901762, (0, 0, 3, 15, 2, 25, 0.005): 837100.3026503037, (0, 0, 3, 15, 2, 25, 0.01): 2515.012467302932, (0, 0, 3, 15, 2, 25, 0.015): 517.7059578463435, (0, 0, 3, 15, 2, 25, 0.02): 258.9202028153336, (0, 0, 3, 15, 2, 25, 0.025): 152.77817483666556, (0, 0, 3, 20, 1, 25, None): 1321.0262700487262, (0, 0, 3, 20, 1, 25, 0.005): 344822.1126813818, (0, 0, 3, 20, 1, 25, 0.01): 1801.901222485989, (0, 0, 3, 20, 1, 25, 0.015): 219.26956068464, (0, 0, 3, 20, 1, 25, 0.02): 155.25239442828092, (0, 0, 3, 20, 1, 25, 0.025): 111.84284723750139, (0, 0, 3, 20, 2, 25, None): 1517.8034757260964, (0, 0, 3, 20, 2, 25, 0.005): 654182.4564537681, (0, 0, 3, 20, 2, 25, 0.01): 2469.689407791281, (0, 0, 3, 20, 2, 25, 0.015): 247.83193101616288, (0, 0, 3, 20, 2, 25, 0.02): 217.4586722770249, (0, 0, 3, 20, 2, 25, 0.025): 160.56049602965254, (0, 1, 0, 15, 1, 25, None): 12199.20981626887, (0, 1, 0, 15, 1, 25, 0.005): 1278793.227665806, (0, 1, 0, 15, 1, 25, 0.01): 16534.219645685575, (0, 1, 0, 15, 1, 25, 0.015): 823.4905629369026, (0, 1, 0, 15, 1, 25, 0.02): 110.72131943826862, (0, 1, 0, 15, 1, 25, 0.025): 23.67224056369282, (0, 1, 0, 15, 2, 25, None): 12472.115761097772, (0, 1, 0, 15, 2, 25, 0.005): 2149491.819514287, (0, 1, 0, 15, 2, 25, 0.01): 23066.8017920083, (0, 1, 0, 15, 2, 25, 0.015): 853.001756097571, (0, 1, 0, 15, 2, 25, 0.02): 141.2844679886002, (0, 1, 0, 15, 2, 25, 0.025): 31.763030182024362, (0, 1, 0, 20, 1, 25, None): 7834.178682934758, (0, 1, 0, 20, 1, 25, 0.005): 1433529.5555428257, (0, 1, 0, 20, 1, 25, 0.01): 15202.140839297754, (0, 1, 0, 20, 1, 25, 0.015): 762.0836043639232, (0, 1, 0, 20, 1, 25, 0.02): 130.44399466285265, (0, 1, 0, 20, 1, 25, 0.025): 14.964448259683849, (0, 1, 0, 20, 2, 25, None): 23026.07080826545, (0, 1, 0, 20, 2, 25, 0.005): 1966826.4133861924, (0, 1, 0, 20, 2, 25, 0.01): 17140.32300843376, (0, 1, 0, 20, 2, 25, 0.015): 1748.4242545051327, (0, 1, 0, 20, 2, 25, 0.02): 148.52432834726864, (0, 1, 0, 20, 2, 25, 0.025): 16.080831620540806, (0, 1, 1, 15, 1, 25, None): 2504.352368072516, (0, 1, 1, 15, 1, 25, 0.005): 663757.0311769233, (0, 1, 1, 15, 1, 25, 0.01): 2486.355211296634, (0, 1, 1, 15, 1, 25, 0.015): 239.6862055355378, (0, 1, 1, 15, 1, 25, 0.02): 179.61531489171705, (0, 1, 1, 15, 1, 25, 0.025): 122.41757955775998, (0, 1, 1, 15, 2, 25, None): 1864.2262546798127, (0, 1, 1, 15, 2, 25, 0.005): 768233.8313812775, (0, 1, 1, 15, 2, 25, 0.01): 3165.1619956007803, (0, 1, 1, 15, 2, 25, 0.015): 243.53441844137592, (0, 1, 1, 15, 2, 25, 0.02): 217.3531431909512, (0, 1, 1, 15, 2, 25, 0.025): 135.41645789229335, (0, 1, 1, 20, 1, 25, None): 1999.0241052677713, (0, 1, 1, 20, 1, 25, 0.005): 713860.4015520852, (0, 1, 1, 20, 1, 25, 0.01): 2503.06798258768, (0, 1, 1, 20, 1, 25, 0.015): 164.45146618612972, (0, 1, 1, 20, 1, 25, 0.02): 141.46070603218848, (0, 1, 1, 20, 1, 25, 0.025): 99.29968681210615, (0, 1, 1, 20, 2, 25, None): 1840.3411457277068, (0, 1, 1, 20, 2, 25, 0.005): 975473.2330197804, (0, 1, 1, 20, 2, 25, 0.01): 3579.2778270944796, (0, 1, 1, 20, 2, 25, 0.015): 174.23097218250174, (0, 1, 1, 20, 2, 25, 0.02): 210.54840980111558, (0, 1, 1, 20, 2, 25, 0.025): 99.73053253039978, (0, 1, 2, 15, 1, 25, None): 2236.008023040314, (0, 1, 2, 15, 1, 25, 0.005): 903535.6911196823, (0, 1, 2, 15, 1, 25, 0.01): 4461.907979426946, (0, 1, 2, 15, 1, 25, 0.015): 178.0420591620835, (0, 1, 2, 15, 1, 25, 0.02): 121.07629260822554, (0, 1, 2, 15, 1, 25, 0.025): 90.79218124923634, (0, 1, 2, 15, 2, 25, None): 3045.520556543651, (0, 1, 2, 15, 2, 25, 0.005): 1261702.5316892932, (0, 1, 2, 15, 2, 25, 0.01): 5979.963008571856, (0, 1, 2, 15, 2, 25, 0.015): 205.82807940558558, (0, 1, 2, 15, 2, 25, 0.02): 109.63265045127503, (0, 1, 2, 15, 2, 25, 0.025): 96.82950554035418, (0, 1, 2, 20, 1, 25, None): 3216.826309859707, (0, 1, 2, 20, 1, 25, 0.005): 984289.4829577516, (0, 1, 2, 20, 1, 25, 0.01): 4952.807279918551, (0, 1, 2, 20, 1, 25, 0.015): 87.88932199906453, (0, 1, 2, 20, 1, 25, 0.02): 62.36251376471388, (0, 1, 2, 20, 1, 25, 0.025): 48.445772157817615, (0, 1, 2, 20, 2, 25, None): 2865.787311186999, (0, 1, 2, 20, 2, 25, 0.005): 1347698.1895511062, (0, 1, 2, 20, 2, 25, 0.01): 5518.748526607447, (0, 1, 2, 20, 2, 25, 0.015): 167.2197895070491, (0, 1, 2, 20, 2, 25, 0.02): 92.8003650240832, (0, 1, 2, 20, 2, 25, 0.025): 63.44812524622645, (0, 1, 3, 15, 1, 25, None): 1280.3081322774165, (0, 1, 3, 15, 1, 25, 0.005): 399711.30148183793, (0, 1, 3, 15, 1, 25, 0.01): 1639.0496878287984, (0, 1, 3, 15, 1, 25, 0.015): 325.0322486549896, (0, 1, 3, 15, 1, 25, 0.02): 225.90228514383682, (0, 1, 3, 15, 1, 25, 0.025): 163.32673521513274, (0, 1, 3, 15, 2, 25, None): 1706.5177195704107, (0, 1, 3, 15, 2, 25, 0.005): 758816.0986066384, (0, 1, 3, 15, 2, 25, 0.01): 2551.0993987267775, (0, 1, 3, 15, 2, 25, 0.015): 379.40504891263026, (0, 1, 3, 15, 2, 25, 0.02): 252.65669109086957, (0, 1, 3, 15, 2, 25, 0.025): 163.69535150103005, (0, 1, 3, 20, 1, 25, None): 1375.8124644068234, (0, 1, 3, 20, 1, 25, 0.005): 452293.424438072, (0, 1, 3, 20, 1, 25, 0.01): 1738.7028635126449, (0, 1, 3, 20, 1, 25, 0.015): 250.8871734405317, (0, 1, 3, 20, 1, 25, 0.02): 202.286027781557, (0, 1, 3, 20, 1, 25, 0.025): 121.18038814143307, (0, 1, 3, 20, 2, 25, None): 1666.6271003419997, (0, 1, 3, 20, 2, 25, 0.005): 541720.9015446767, (0, 1, 3, 20, 2, 25, 0.01): 2380.8747947999855, (0, 1, 3, 20, 2, 25, 0.015): 335.08721554470384, (0, 1, 3, 20, 2, 25, 0.02): 223.22691846647905, (0, 1, 3, 20, 2, 25, 0.025): 151.63056184999053, (0, 2, 0, 15, 1, 25, None): 10811.713688333308, (0, 2, 0, 15, 1, 25, 0.005): 1285553.0711647875, (0, 2, 0, 15, 1, 25, 0.01): 13279.049310866843, (0, 2, 0, 15, 1, 25, 0.015): 976.0045769433875, (0, 2, 0, 15, 1, 25, 0.02): 117.8702444627842, (0, 2, 0, 15, 1, 25, 0.025): 22.969492698025746, (0, 2, 0, 15, 2, 25, None): 17718.970181511246, (0, 2, 0, 15, 2, 25, 0.005): 1538386.1862657033, (0, 2, 0, 15, 2, 25, 0.01): 17657.376295405546, (0, 2, 0, 15, 2, 25, 0.015): 905.0735655455311, (0, 2, 0, 15, 2, 25, 0.02): 173.64045397752028, (0, 2, 0, 15, 2, 25, 0.025): 39.6232203207889, (0, 2, 0, 20, 1, 25, None): 7845.781162658136, (0, 2, 0, 20, 1, 25, 0.005): 1404855.3363640225, (0, 2, 0, 20, 1, 25, 0.01): 13630.272596621488, (0, 2, 0, 20, 1, 25, 0.015): 856.6685379463626, (0, 2, 0, 20, 1, 25, 0.02): 79.37793298386974, (0, 2, 0, 20, 1, 25, 0.025): 19.116505565534485, (0, 2, 0, 20, 2, 25, None): 18529.7472367422, (0, 2, 0, 20, 2, 25, 0.005): 2373172.011475519, (0, 2, 0, 20, 2, 25, 0.01): 21665.29629537753, (0, 2, 0, 20, 2, 25, 0.015): 1232.3669949181717, (0, 2, 0, 20, 2, 25, 0.02): 94.3995326431774, (0, 2, 0, 20, 2, 25, 0.025): 18.343969544026375, (0, 2, 1, 15, 1, 25, None): 1014.3781829763271, (0, 2, 1, 15, 1, 25, 0.005): 508067.77011786413, (0, 2, 1, 15, 1, 25, 0.01): 1952.8756736102384, (0, 2, 1, 15, 1, 25, 0.015): 188.41125701822764, (0, 2, 1, 15, 1, 25, 0.02): 197.3302622891744, (0, 2, 1, 15, 1, 25, 0.025): 107.97076730961649, (0, 2, 1, 15, 2, 25, None): 1915.0435808183274, (0, 2, 1, 15, 2, 25, 0.005): 836239.945041848, (0, 2, 1, 15, 2, 25, 0.01): 3433.7343114358523, (0, 2, 1, 15, 2, 25, 0.015): 351.4865988564448, (0, 2, 1, 15, 2, 25, 0.02): 178.29470667587623, (0, 2, 1, 15, 2, 25, 0.025): 131.3804599808157, (0, 2, 1, 20, 1, 25, None): 1881.3305840242936, (0, 2, 1, 20, 1, 25, 0.005): 560724.4678440716, (0, 2, 1, 20, 1, 25, 0.01): 2756.9678249256863, (0, 2, 1, 20, 1, 25, 0.015): 158.96379591420984, (0, 2, 1, 20, 1, 25, 0.02): 117.42784372348999, (0, 2, 1, 20, 1, 25, 0.025): 106.84766828545507, (0, 2, 1, 20, 2, 25, None): 2593.0088889237345, (0, 2, 1, 20, 2, 25, 0.005): 1008904.1369227687, (0, 2, 1, 20, 2, 25, 0.01): 4150.177859980313, (0, 2, 1, 20, 2, 25, 0.015): 184.60707343512675, (0, 2, 1, 20, 2, 25, 0.02): 128.74791538429938, (0, 2, 1, 20, 2, 25, 0.025): 139.67259616004827, (0, 2, 2, 15, 1, 25, None): 2779.612435637752, (0, 2, 2, 15, 1, 25, 0.005): 982272.7973120156, (0, 2, 2, 15, 1, 25, 0.01): 5105.941283804636, (0, 2, 2, 15, 1, 25, 0.015): 177.64810712155867, (0, 2, 2, 15, 1, 25, 0.02): 112.27588443209544, (0, 2, 2, 15, 1, 25, 0.025): 80.8459808125975, (0, 2, 2, 15, 2, 25, None): 3823.952566816023, (0, 2, 2, 15, 2, 25, 0.005): 1259058.279577526, (0, 2, 2, 15, 2, 25, 0.01): 5158.3987689659725, (0, 2, 2, 15, 2, 25, 0.015): 194.77771885012535, (0, 2, 2, 15, 2, 25, 0.02): 126.65692275008752, (0, 2, 2, 15, 2, 25, 0.025): 113.57428132813627, (0, 2, 2, 20, 1, 25, None): 2382.4163523562634, (0, 2, 2, 20, 1, 25, 0.005): 1009069.4311913714, (0, 2, 2, 20, 1, 25, 0.01): 4603.916865065595, (0, 2, 2, 20, 1, 25, 0.015): 109.81919868764871, (0, 2, 2, 20, 1, 25, 0.02): 85.02397079406073, (0, 2, 2, 20, 1, 25, 0.025): 77.65787362850283, (0, 2, 2, 20, 2, 25, None): 4270.1291307679, (0, 2, 2, 20, 2, 25, 0.005): 1083329.2443912409, (0, 2, 2, 20, 2, 25, 0.01): 5649.737901346357, (0, 2, 2, 20, 2, 25, 0.015): 141.38250606367237, (0, 2, 2, 20, 2, 25, 0.02): 78.40997713977234, (0, 2, 2, 20, 2, 25, 0.025): 85.62009987132224, (0, 2, 3, 15, 1, 25, None): 890.3829134870156, (0, 2, 3, 15, 1, 25, 0.005): 611654.683299303, (0, 2, 3, 15, 1, 25, 0.01): 1698.365352545945, (0, 2, 3, 15, 1, 25, 0.015): 349.02365073683933, (0, 2, 3, 15, 1, 25, 0.02): 188.0360580937762, (0, 2, 3, 15, 1, 25, 0.025): 155.44212541003876, (0, 2, 3, 15, 2, 25, None): 1719.805985450185, (0, 2, 3, 15, 2, 25, 0.005): 754974.5216512628, (0, 2, 3, 15, 2, 25, 0.01): 2552.561832788978, (0, 2, 3, 15, 2, 25, 0.015): 425.3547305125522, (0, 2, 3, 15, 2, 25, 0.02): 255.68480435816667, (0, 2, 3, 15, 2, 25, 0.025): 199.44965421219396, (0, 2, 3, 20, 1, 25, None): 1364.3722008837544, (0, 2, 3, 20, 1, 25, 0.005): 501175.76793290226, (0, 2, 3, 20, 1, 25, 0.01): 2253.8532516273885, (0, 2, 3, 20, 1, 25, 0.015): 205.49643886369617, (0, 2, 3, 20, 1, 25, 0.02): 229.03213412591057, (0, 2, 3, 20, 1, 25, 0.025): 107.48546467163939, (0, 2, 3, 20, 2, 25, None): 1909.3950141407531, (0, 2, 3, 20, 2, 25, 0.005): 822639.9145236933, (0, 2, 3, 20, 2, 25, 0.01): 2203.782441895363, (0, 2, 3, 20, 2, 25, 0.015): 311.97051107657717, (0, 2, 3, 20, 2, 25, 0.02): 175.3879779203202, (0, 2, 3, 20, 2, 25, 0.025): 171.37532681208745, (1, 0, 0, 15, 1, 25, None): 20997.243298444526, (1, 0, 0, 15, 1, 25, 0.005): 2282183.0237840824, (1, 0, 0, 15, 1, 25, 0.01): 33676.65279360104, (1, 0, 0, 15, 1, 25, 0.015): 2862.3931938065284, (1, 0, 0, 15, 1, 25, 0.02): 228.32570083886588, (1, 0, 0, 15, 1, 25, 0.025): 56.34028852882536, (1, 0, 0, 15, 2, 25, None): 33064.025936132246, (1, 0, 0, 15, 2, 25, 0.005): 3365206.158169336, (1, 0, 0, 15, 2, 25, 0.01): 46881.06181760663, (1, 0, 0, 15, 2, 25, 0.015): 2748.6268147832884, (1, 0, 0, 15, 2, 25, 0.02): 372.81495200195934, (1, 0, 0, 15, 2, 25, 0.025): 63.45068210320754, (1, 0, 0, 20, 1, 25, None): 17629.833898596757, (1, 0, 0, 20, 1, 25, 0.005): 2120997.7630259134, (1, 0, 0, 20, 1, 25, 0.01): 42304.62016518967, (1, 0, 0, 20, 1, 25, 0.015): 2255.5391540650435, (1, 0, 0, 20, 1, 25, 0.02): 202.11114516455447, (1, 0, 0, 20, 1, 25, 0.025): 47.9719028642101, (1, 0, 0, 20, 2, 25, None): 18027.066429041326, (1, 0, 0, 20, 2, 25, 0.005): 3723759.926880531, (1, 0, 0, 20, 2, 25, 0.01): 37068.62568323754, (1, 0, 0, 20, 2, 25, 0.015): 2875.9016697724783, (1, 0, 0, 20, 2, 25, 0.02): 356.3516850159529, (1, 0, 0, 20, 2, 25, 0.025): 58.7913771785476, (1, 0, 1, 15, 1, 25, None): 2575.8488478909107, (1, 0, 1, 15, 1, 25, 0.005): 601462.6698488252, (1, 0, 1, 15, 1, 25, 0.01): 3296.205276043899, (1, 0, 1, 15, 1, 25, 0.015): 504.3484628561396, (1, 0, 1, 15, 1, 25, 0.02): 433.62308573274703, (1, 0, 1, 15, 1, 25, 0.025): 292.80222490051256, (1, 0, 1, 15, 2, 25, None): 1811.0211799666458, (1, 0, 1, 15, 2, 25, 0.005): 1063087.123845055, (1, 0, 1, 15, 2, 25, 0.01): 5303.789503474068, (1, 0, 1, 15, 2, 25, 0.015): 719.4220019159277, (1, 0, 1, 15, 2, 25, 0.02): 493.87416599007304, (1, 0, 1, 15, 2, 25, 0.025): 350.5772215441772, (1, 0, 1, 20, 1, 25, None): 2105.36267772054, (1, 0, 1, 20, 1, 25, 0.005): 768371.1196808426, (1, 0, 1, 20, 1, 25, 0.01): 4534.987622384513, (1, 0, 1, 20, 1, 25, 0.015): 454.92456186188747, (1, 0, 1, 20, 1, 25, 0.02): 419.22740654716256, (1, 0, 1, 20, 1, 25, 0.025): 362.0622815672982, (1, 0, 1, 20, 2, 25, None): 3457.2191402847398, (1, 0, 1, 20, 2, 25, 0.005): 1268207.5117792694, (1, 0, 1, 20, 2, 25, 0.01): 4962.0911911574785, (1, 0, 1, 20, 2, 25, 0.015): 588.3156474180358, (1, 0, 1, 20, 2, 25, 0.02): 515.5856780495393, (1, 0, 1, 20, 2, 25, 0.025): 312.98018913315656, (1, 0, 2, 15, 1, 25, None): 2431.9763007425963, (1, 0, 2, 15, 1, 25, 0.005): 1079562.4987431145, (1, 0, 2, 15, 1, 25, 0.01): 5179.9969773460925, (1, 0, 2, 15, 1, 25, 0.015): 378.7366850340298, (1, 0, 2, 15, 1, 25, 0.02): 298.022999726156, (1, 0, 2, 15, 1, 25, 0.025): 239.63296472054813, (1, 0, 2, 15, 2, 25, None): 2260.5451499815313, (1, 0, 2, 15, 2, 25, 0.005): 1766718.2952934904, (1, 0, 2, 15, 2, 25, 0.01): 7132.750471204352, (1, 0, 2, 15, 2, 25, 0.015): 488.93539513207764, (1, 0, 2, 15, 2, 25, 0.02): 330.7725088846759, (1, 0, 2, 15, 2, 25, 0.025): 304.9881977100498, (1, 0, 2, 20, 1, 25, None): 2405.6076709657714, (1, 0, 2, 20, 1, 25, 0.005): 901665.5518834552, (1, 0, 2, 20, 1, 25, 0.01): 7073.358691078914, (1, 0, 2, 20, 1, 25, 0.015): 349.7197897150543, (1, 0, 2, 20, 1, 25, 0.02): 267.3259686018571, (1, 0, 2, 20, 1, 25, 0.025): 214.86944441801577, (1, 0, 2, 20, 2, 25, None): 3092.982910014819, (1, 0, 2, 20, 2, 25, 0.005): 1590440.0639999055, (1, 0, 2, 20, 2, 25, 0.01): 7692.969594750401, (1, 0, 2, 20, 2, 25, 0.015): 366.967578078123, (1, 0, 2, 20, 2, 25, 0.02): 285.7157639285531, (1, 0, 2, 20, 2, 25, 0.025): 258.47512308140904, (1, 0, 3, 15, 1, 25, None): 1933.295247148594, (1, 0, 3, 15, 1, 25, 0.005): 675272.0053967386, (1, 0, 3, 15, 1, 25, 0.01): 2529.598416971229, (1, 0, 3, 15, 1, 25, 0.015): 780.0527046567605, (1, 0, 3, 15, 1, 25, 0.02): 515.6324768084895, (1, 0, 3, 15, 1, 25, 0.025): 376.84481419642583, (1, 0, 3, 15, 2, 25, None): 1658.1744076260645, (1, 0, 3, 15, 2, 25, 0.005): 981171.9835537255, (1, 0, 3, 15, 2, 25, 0.01): 3097.8991712706984, (1, 0, 3, 15, 2, 25, 0.015): 1074.5648788566107, (1, 0, 3, 15, 2, 25, 0.02): 549.9076869472269, (1, 0, 3, 15, 2, 25, 0.025): 433.9047198200598, (1, 0, 3, 20, 1, 25, None): 1243.8819574921147, (1, 0, 3, 20, 1, 25, 0.005): 596897.6247314212, (1, 0, 3, 20, 1, 25, 0.01): 2308.121439401734, (1, 0, 3, 20, 1, 25, 0.015): 677.1068619151489, (1, 0, 3, 20, 1, 25, 0.02): 441.5942470205077, (1, 0, 3, 20, 1, 25, 0.025): 301.97697540237675, (1, 0, 3, 20, 2, 25, None): 2202.096009193115, (1, 0, 3, 20, 2, 25, 0.005): 813863.6204521703, (1, 0, 3, 20, 2, 25, 0.01): 3116.4610409635075, (1, 0, 3, 20, 2, 25, 0.015): 710.7223734347581, (1, 0, 3, 20, 2, 25, 0.02): 558.4403395002163, (1, 0, 3, 20, 2, 25, 0.025): 317.88306234292463, (1, 1, 0, 15, 1, 25, None): 23469.700727316358, (1, 1, 0, 15, 1, 25, 0.005): 2961291.59159126, (1, 1, 0, 15, 1, 25, 0.01): 26008.585309015732, (1, 1, 0, 15, 1, 25, 0.015): 2219.789253490097, (1, 1, 0, 15, 1, 25, 0.02): 195.7638652531944, (1, 1, 0, 15, 1, 25, 0.025): 54.149060878196536, (1, 1, 0, 15, 2, 25, None): 44830.33631433357, (1, 1, 0, 15, 2, 25, 0.005): 3753568.1824906943, (1, 1, 0, 15, 2, 25, 0.01): 48827.78636557979, (1, 1, 0, 15, 2, 25, 0.015): 2432.612173800835, (1, 1, 0, 15, 2, 25, 0.02): 250.99230181068282, (1, 1, 0, 15, 2, 25, 0.025): 58.95435704090903, (1, 1, 0, 20, 1, 25, None): 23251.14948912171, (1, 1, 0, 20, 1, 25, 0.005): 3132092.587982708, (1, 1, 0, 20, 1, 25, 0.01): 36562.80447842994, (1, 1, 0, 20, 1, 25, 0.015): 2224.641277819724, (1, 1, 0, 20, 1, 25, 0.02): 307.11140077368884, (1, 1, 0, 20, 1, 25, 0.025): 48.47890442965514, (1, 1, 0, 20, 2, 25, None): 29327.081370919943, (1, 1, 0, 20, 2, 25, 0.005): 4323773.556686159, (1, 1, 0, 20, 2, 25, 0.01): 49364.955561714974, (1, 1, 0, 20, 2, 25, 0.015): 2533.116307484561, (1, 1, 0, 20, 2, 25, 0.02): 293.46560171456133, (1, 1, 0, 20, 2, 25, 0.025): 55.08848168508768, (1, 1, 1, 15, 1, 25, None): 1674.5473331246624, (1, 1, 1, 15, 1, 25, 0.005): 715344.4502888069, (1, 1, 1, 15, 1, 25, 0.01): 2875.583248041754, (1, 1, 1, 15, 1, 25, 0.015): 610.9381745565227, (1, 1, 1, 15, 1, 25, 0.02): 418.138289691488, (1, 1, 1, 15, 1, 25, 0.025): 332.6544416956479, (1, 1, 1, 15, 2, 25, None): 2898.0700297267563, (1, 1, 1, 15, 2, 25, 0.005): 1268608.4631941405, (1, 1, 1, 15, 2, 25, 0.01): 3673.432378308276, (1, 1, 1, 15, 2, 25, 0.015): 658.4845771640619, (1, 1, 1, 15, 2, 25, 0.02): 481.604590667299, (1, 1, 1, 15, 2, 25, 0.025): 363.72756561382323, (1, 1, 1, 20, 1, 25, None): 2674.051008129202, (1, 1, 1, 20, 1, 25, 0.005): 758677.1216812719, (1, 1, 1, 20, 1, 25, 0.01): 3438.710680200179, (1, 1, 1, 20, 1, 25, 0.015): 464.1862400113795, (1, 1, 1, 20, 1, 25, 0.02): 403.49371359118794, (1, 1, 1, 20, 1, 25, 0.025): 256.70781160741757, (1, 1, 1, 20, 2, 25, None): 2573.80566628848, (1, 1, 1, 20, 2, 25, 0.005): 897606.8550854445, (1, 1, 1, 20, 2, 25, 0.01): 4458.842464773639, (1, 1, 1, 20, 2, 25, 0.015): 652.4500567353034, (1, 1, 1, 20, 2, 25, 0.02): 448.1887110532437, (1, 1, 1, 20, 2, 25, 0.025): 319.5038899806632, (1, 1, 2, 15, 1, 25, None): 2501.1129047766344, (1, 1, 2, 15, 1, 25, 0.005): 1105061.035468784, (1, 1, 2, 15, 1, 25, 0.01): 5906.8977701634985, (1, 1, 2, 15, 1, 25, 0.015): 420.7419265447324, (1, 1, 2, 15, 1, 25, 0.02): 301.92621665129195, (1, 1, 2, 15, 1, 25, 0.025): 261.6713543622441, (1, 1, 2, 15, 2, 25, None): 3802.311283829459, (1, 1, 2, 15, 2, 25, 0.005): 1225821.9648107572, (1, 1, 2, 15, 2, 25, 0.01): 6756.824575323151, (1, 1, 2, 15, 2, 25, 0.015): 447.66087825916395, (1, 1, 2, 15, 2, 25, 0.02): 367.3625258437856, (1, 1, 2, 15, 2, 25, 0.025): 305.8465546847141, (1, 1, 2, 20, 1, 25, None): 3366.4077699045097, (1, 1, 2, 20, 1, 25, 0.005): 1077713.7653767392, (1, 1, 2, 20, 1, 25, 0.01): 6549.800623552149, (1, 1, 2, 20, 1, 25, 0.015): 350.10417997987287, (1, 1, 2, 20, 1, 25, 0.02): 277.6429832882896, (1, 1, 2, 20, 1, 25, 0.025): 238.2384001677454, (1, 1, 2, 20, 2, 25, None): 2959.596273574043, (1, 1, 2, 20, 2, 25, 0.005): 1923679.8353616507, (1, 1, 2, 20, 2, 25, 0.01): 6689.26578211513, (1, 1, 2, 20, 2, 25, 0.015): 425.0490214876404, (1, 1, 2, 20, 2, 25, 0.02): 300.2289070294703, (1, 1, 2, 20, 2, 25, 0.025): 246.38801674589743, (1, 1, 3, 15, 1, 25, None): 1475.8824036128763, (1, 1, 3, 15, 1, 25, 0.005): 571195.7812662765, (1, 1, 3, 15, 1, 25, 0.01): 2725.664272295195, (1, 1, 3, 15, 1, 25, 0.015): 754.0758900675993, (1, 1, 3, 15, 1, 25, 0.02): 478.85200500232327, (1, 1, 3, 15, 1, 25, 0.025): 352.78248718756356, (1, 1, 3, 15, 2, 25, None): 1939.5180773639177, (1, 1, 3, 15, 2, 25, 0.005): 599009.808040092, (1, 1, 3, 15, 2, 25, 0.01): 3008.7846800566026, (1, 1, 3, 15, 2, 25, 0.015): 844.3803177593411, (1, 1, 3, 15, 2, 25, 0.02): 678.7493036704741, (1, 1, 3, 15, 2, 25, 0.025): 452.21920289860583, (1, 1, 3, 20, 1, 25, None): 1612.8544914548243, (1, 1, 3, 20, 1, 25, 0.005): 582474.8062297388, (1, 1, 3, 20, 1, 25, 0.01): 2617.8389305145006, (1, 1, 3, 20, 1, 25, 0.015): 692.6543168716896, (1, 1, 3, 20, 1, 25, 0.02): 480.97004947154426, (1, 1, 3, 20, 1, 25, 0.025): 322.7490559119352, (1, 1, 3, 20, 2, 25, None): 1766.650692760762, (1, 1, 3, 20, 2, 25, 0.005): 939157.7975647267, (1, 1, 3, 20, 2, 25, 0.01): 2885.6011859282526, (1, 1, 3, 20, 2, 25, 0.015): 719.6864494485002, (1, 1, 3, 20, 2, 25, 0.02): 602.8156944162588, (1, 1, 3, 20, 2, 25, 0.025): 357.7487687344304, (1, 2, 0, 15, 1, 25, None): 10711.1256849645, (1, 2, 0, 15, 1, 25, 0.005): 2564826.2871371517, (1, 2, 0, 15, 1, 25, 0.01): 33593.52370133726, (1, 2, 0, 15, 1, 25, 0.015): 2239.4470032115437, (1, 2, 0, 15, 1, 25, 0.02): 286.1397393844104, (1, 2, 0, 15, 1, 25, 0.025): 59.533208418951226, (1, 2, 0, 15, 2, 25, None): 17445.23347818817, (1, 2, 0, 15, 2, 25, 0.005): 3058071.0896253255, (1, 2, 0, 15, 2, 25, 0.01): 57987.28275548156, (1, 2, 0, 15, 2, 25, 0.015): 2356.4800864303143, (1, 2, 0, 15, 2, 25, 0.02): 314.5911675877718, (1, 2, 0, 15, 2, 25, 0.025): 61.46859644705417, (1, 2, 0, 20, 1, 25, None): 17465.97517834531, (1, 2, 0, 20, 1, 25, 0.005): 1868297.5304398346, (1, 2, 0, 20, 1, 25, 0.01): 42190.50336588186, (1, 2, 0, 20, 1, 25, 0.015): 2151.4261007194045, (1, 2, 0, 20, 1, 25, 0.02): 228.49965957920037, (1, 2, 0, 20, 1, 25, 0.025): 57.00058911707792, (1, 2, 0, 20, 2, 25, None): 24541.709344573042, (1, 2, 0, 20, 2, 25, 0.005): 3213552.814826232, (1, 2, 0, 20, 2, 25, 0.01): 44462.12320140706, (1, 2, 0, 20, 2, 25, 0.015): 3454.1252970310907, (1, 2, 0, 20, 2, 25, 0.02): 305.40530249520174, (1, 2, 0, 20, 2, 25, 0.025): 64.84503875807606, (1, 2, 1, 15, 1, 25, None): 1795.2016366791916, (1, 2, 1, 15, 1, 25, 0.005): 782982.8395896398, (1, 2, 1, 15, 1, 25, 0.01): 2881.587439077028, (1, 2, 1, 15, 1, 25, 0.015): 589.5561133801438, (1, 2, 1, 15, 1, 25, 0.02): 496.0070708734323, (1, 2, 1, 15, 1, 25, 0.025): 335.6434728217937, (1, 2, 1, 15, 2, 25, None): 2897.192053025296, (1, 2, 1, 15, 2, 25, 0.005): 1375365.8236973023, (1, 2, 1, 15, 2, 25, 0.01): 4873.661547816528, (1, 2, 1, 15, 2, 25, 0.015): 681.9175824694431, (1, 2, 1, 15, 2, 25, 0.02): 461.90663520722717, (1, 2, 1, 15, 2, 25, 0.025): 423.8729922994819, (1, 2, 1, 20, 1, 25, None): 1757.1117796080962, (1, 2, 1, 20, 1, 25, 0.005): 674398.1264503752, (1, 2, 1, 20, 1, 25, 0.01): 3208.0930708487144, (1, 2, 1, 20, 1, 25, 0.015): 511.60029699729546, (1, 2, 1, 20, 1, 25, 0.02): 298.9308807695353, (1, 2, 1, 20, 1, 25, 0.025): 298.20893887202476, (1, 2, 1, 20, 2, 25, None): 3183.804007063581, (1, 2, 1, 20, 2, 25, 0.005): 1148839.342841423, (1, 2, 1, 20, 2, 25, 0.01): 4885.475443330353, (1, 2, 1, 20, 2, 25, 0.015): 594.111830016536, (1, 2, 1, 20, 2, 25, 0.02): 468.28223263988576, (1, 2, 1, 20, 2, 25, 0.025): 341.88679227077284, (1, 2, 2, 15, 1, 25, None): 3735.326066486406, (1, 2, 2, 15, 1, 25, 0.005): 990677.2824268363, (1, 2, 2, 15, 1, 25, 0.01): 5372.900435458249, (1, 2, 2, 15, 1, 25, 0.015): 425.33106447481464, (1, 2, 2, 15, 1, 25, 0.02): 305.72822376595866, (1, 2, 2, 15, 1, 25, 0.025): 221.52970791426023, (1, 2, 2, 15, 2, 25, None): 2700.1989213193183, (1, 2, 2, 15, 2, 25, 0.005): 1753134.7700469545, (1, 2, 2, 15, 2, 25, 0.01): 7832.101240621902, (1, 2, 2, 15, 2, 25, 0.015): 474.24210841171595, (1, 2, 2, 15, 2, 25, 0.02): 371.9024690558695, (1, 2, 2, 15, 2, 25, 0.025): 299.7146195249328, (1, 2, 2, 20, 1, 25, None): 1973.8825073123376, (1, 2, 2, 20, 1, 25, 0.005): 939291.3150921465, (1, 2, 2, 20, 1, 25, 0.01): 5603.018727802137, (1, 2, 2, 20, 1, 25, 0.015): 318.2530874762219, (1, 2, 2, 20, 1, 25, 0.02): 307.573393109218, (1, 2, 2, 20, 1, 25, 0.025): 197.94252947016628, (1, 2, 2, 20, 2, 25, None): 3211.6535849375787, (1, 2, 2, 20, 2, 25, 0.005): 1778371.2533026566, (1, 2, 2, 20, 2, 25, 0.01): 8617.113047796813, (1, 2, 2, 20, 2, 25, 0.015): 423.43597719907325, (1, 2, 2, 20, 2, 25, 0.02): 346.6544113962551, (1, 2, 2, 20, 2, 25, 0.025): 269.5492235135025, (1, 2, 3, 15, 1, 25, None): 1451.714677968888, (1, 2, 3, 15, 1, 25, 0.005): 547995.7942654245, (1, 2, 3, 15, 1, 25, 0.01): 2850.910979689806, (1, 2, 3, 15, 1, 25, 0.015): 822.2707353231597, (1, 2, 3, 15, 1, 25, 0.02): 449.4717021269909, (1, 2, 3, 15, 1, 25, 0.025): 328.13909326686394, (1, 2, 3, 15, 2, 25, None): 2232.266817152677, (1, 2, 3, 15, 2, 25, 0.005): 733744.1317292429, (1, 2, 3, 15, 2, 25, 0.01): 2699.5324033661273, (1, 2, 3, 15, 2, 25, 0.015): 899.6866877907361, (1, 2, 3, 15, 2, 25, 0.02): 739.364099389619, (1, 2, 3, 15, 2, 25, 0.025): 466.9953016260226, (1, 2, 3, 20, 1, 25, None): 1584.6561467628753, (1, 2, 3, 20, 1, 25, 0.005): 547320.6882663555, (1, 2, 3, 20, 1, 25, 0.01): 3321.603460405748, (1, 2, 3, 20, 1, 25, 0.015): 623.2278052395789, (1, 2, 3, 20, 1, 25, 0.02): 456.9681561881181, (1, 2, 3, 20, 1, 25, 0.025): 314.62981696771874, (1, 2, 3, 20, 2, 25, None): 1959.2845619956554, (1, 2, 3, 20, 2, 25, 0.005): 1066276.3152376872, (1, 2, 3, 20, 2, 25, 0.01): 4116.101056110349, (1, 2, 3, 20, 2, 25, 0.015): 785.3734131640591, (1, 2, 3, 20, 2, 25, 0.02): 600.0622230833737, (1, 2, 3, 20, 2, 25, 0.025): 420.73745940137525, (2, 0, 0, 15, 1, 25, None): 24797182.257168543, (2, 0, 0, 15, 1, 25, 0.005): 7035303.773940083, (2, 0, 0, 15, 1, 25, 0.01): 137369.2038710003, (2, 0, 0, 15, 1, 25, 0.015): 10470.117723993022, (2, 0, 0, 15, 1, 25, 0.02): 1077.4578012734125, (2, 0, 0, 15, 1, 25, 0.025): 53.97801715786118, (2, 0, 0, 15, 2, 25, None): 30477205.67902853, (2, 0, 0, 15, 2, 25, 0.005): 8765123.014551869, (2, 0, 0, 15, 2, 25, 0.01): 179549.00853355735, (2, 0, 0, 15, 2, 25, 0.015): 13117.52845591953, (2, 0, 0, 15, 2, 25, 0.02): 1090.7858608216159, (2, 0, 0, 15, 2, 25, 0.025): 56.78827760739065, (2, 0, 0, 20, 1, 25, None): 45800904.33123926, (2, 0, 0, 20, 1, 25, 0.005): 6641101.234090248, (2, 0, 0, 20, 1, 25, 0.01): 177638.10484963792, (2, 0, 0, 20, 1, 25, 0.015): 13933.293656191889, (2, 0, 0, 20, 1, 25, 0.02): 1136.9181490130945, (2, 0, 0, 20, 1, 25, 0.025): 66.10680944732664, (2, 0, 0, 20, 2, 25, None): 43517969.77592028, (2, 0, 0, 20, 2, 25, 0.005): 8610079.03484498, (2, 0, 0, 20, 2, 25, 0.01): 211955.31765650565, (2, 0, 0, 20, 2, 25, 0.015): 15181.940198673858, (2, 0, 0, 20, 2, 25, 0.02): 1361.4693330141445, (2, 0, 0, 20, 2, 25, 0.025): 88.35884829989958, (2, 0, 1, 15, 1, 25, None): 7487258.691373095, (2, 0, 1, 15, 1, 25, 0.005): 579040.307346278, (2, 0, 1, 15, 1, 25, 0.01): 2179.7106875307627, (2, 0, 1, 15, 1, 25, 0.015): 1915.6390627695869, (2, 0, 1, 15, 1, 25, 0.02): 1635.9436266080197, (2, 0, 1, 15, 1, 25, 0.025): 1312.908928357344, (2, 0, 1, 15, 2, 25, None): 12860918.022822868, (2, 0, 1, 15, 2, 25, 0.005): 824120.0815006071, (2, 0, 1, 15, 2, 25, 0.01): 2016.3233923967875, (2, 0, 1, 15, 2, 25, 0.015): 2379.799624537961, (2, 0, 1, 15, 2, 25, 0.02): 2125.562460521197, (2, 0, 1, 15, 2, 25, 0.025): 1443.3147320429814, (2, 0, 1, 20, 1, 25, None): 8922925.654421382, (2, 0, 1, 20, 1, 25, 0.005): 707234.1649676795, (2, 0, 1, 20, 1, 25, 0.01): 1902.5749727636855, (2, 0, 1, 20, 1, 25, 0.015): 1621.1145484207327, (2, 0, 1, 20, 1, 25, 0.02): 1600.8816240980925, (2, 0, 1, 20, 1, 25, 0.025): 1113.282952042031, (2, 0, 1, 20, 2, 25, None): 4910507.490403443, (2, 0, 1, 20, 2, 25, 0.005): 872293.0014987293, (2, 0, 1, 20, 2, 25, 0.01): 1851.0406557395302, (2, 0, 1, 20, 2, 25, 0.015): 2005.5222318267058, (2, 0, 1, 20, 2, 25, 0.02): 1849.8705516048826, (2, 0, 1, 20, 2, 25, 0.025): 1286.4798510807418, (2, 0, 2, 15, 1, 25, None): 7129682.255687194, (2, 0, 2, 15, 1, 25, 0.005): 1122077.6165612796, (2, 0, 2, 15, 1, 25, 0.01): 5674.963079129163, (2, 0, 2, 15, 1, 25, 0.015): 761.1055332394764, (2, 0, 2, 15, 1, 25, 0.02): 1073.4196719524798, (2, 0, 2, 15, 1, 25, 0.025): 919.7894165693485, (2, 0, 2, 15, 2, 25, None): 12701528.21819957, (2, 0, 2, 15, 2, 25, 0.005): 1376455.8555580082, (2, 0, 2, 15, 2, 25, 0.01): 8036.646799903155, (2, 0, 2, 15, 2, 25, 0.015): 891.3039149891935, (2, 0, 2, 15, 2, 25, 0.02): 1392.5875215824412, (2, 0, 2, 15, 2, 25, 0.025): 1218.5373418317065, (2, 0, 2, 20, 1, 25, None): 3945922.3948769053, (2, 0, 2, 20, 1, 25, 0.005): 1131693.6202526584, (2, 0, 2, 20, 1, 25, 0.01): 6767.342769850836, (2, 0, 2, 20, 1, 25, 0.015): 564.4749437121949, (2, 0, 2, 20, 1, 25, 0.02): 983.1435305341287, (2, 0, 2, 20, 1, 25, 0.025): 781.0769290284762, (2, 0, 2, 20, 2, 25, None): 12702329.394011522, (2, 0, 2, 20, 2, 25, 0.005): 1307192.3958444342, (2, 0, 2, 20, 2, 25, 0.01): 8273.90689281365, (2, 0, 2, 20, 2, 25, 0.015): 665.0490405364625, (2, 0, 2, 20, 2, 25, 0.02): 1174.0253103399837, (2, 0, 2, 20, 2, 25, 0.025): 1026.1728701169893, (2, 0, 3, 15, 1, 25, None): 2630447.329560484, (2, 0, 3, 15, 1, 25, 0.005): 462166.0688833524, (2, 0, 3, 15, 1, 25, 0.01): 2139.879763394986, (2, 0, 3, 15, 1, 25, 0.015): 2850.130585248162, (2, 0, 3, 15, 1, 25, 0.02): 2302.2143781305435, (2, 0, 3, 15, 1, 25, 0.025): 1477.3299479213715, (2, 0, 3, 15, 2, 25, None): 4126156.3104051645, (2, 0, 3, 15, 2, 25, 0.005): 451497.1384302883, (2, 0, 3, 15, 2, 25, 0.01): 2161.0137317010285, (2, 0, 3, 15, 2, 25, 0.015): 3577.7091059285376, (2, 0, 3, 15, 2, 25, 0.02): 2708.409491035739, (2, 0, 3, 15, 2, 25, 0.025): 1815.6822165343604, (2, 0, 3, 20, 1, 25, None): 3022414.83697155, (2, 0, 3, 20, 1, 25, 0.005): 547538.0364254399, (2, 0, 3, 20, 1, 25, 0.01): 1426.960819730714, (2, 0, 3, 20, 1, 25, 0.015): 2562.515514006635, (2, 0, 3, 20, 1, 25, 0.02): 2133.379189937706, (2, 0, 3, 20, 1, 25, 0.025): 1398.2944220465104, (2, 0, 3, 20, 2, 25, None): 4121694.7206519013, (2, 0, 3, 20, 2, 25, 0.005): 539063.5337647118, (2, 0, 3, 20, 2, 25, 0.01): 1927.7746249175277, (2, 0, 3, 20, 2, 25, 0.015): 3234.775143696565, (2, 0, 3, 20, 2, 25, 0.02): 2815.545274885743, (2, 0, 3, 20, 2, 25, 0.025): 1695.5164435713323, (2, 1, 0, 15, 1, 25, None): 32174812.321912102, (2, 1, 0, 15, 1, 25, 0.005): 5387213.786645126, (2, 1, 0, 15, 1, 25, 0.01): 153630.93764070666, (2, 1, 0, 15, 1, 25, 0.015): 11072.080307500102, (2, 1, 0, 15, 1, 25, 0.02): 1095.4648104447872, (2, 1, 0, 15, 1, 25, 0.025): 42.582677409581436, (2, 1, 0, 15, 2, 25, None): 34004346.58387939, (2, 1, 0, 15, 2, 25, 0.005): 7975728.575769226, (2, 1, 0, 15, 2, 25, 0.01): 202012.587104717, (2, 1, 0, 15, 2, 25, 0.015): 12866.304453387778, (2, 1, 0, 15, 2, 25, 0.02): 1228.9352046405327, (2, 1, 0, 15, 2, 25, 0.025): 66.9918919036805, (2, 1, 0, 20, 1, 25, None): 58362146.26939073, (2, 1, 0, 20, 1, 25, 0.005): 7800352.983328146, (2, 1, 0, 20, 1, 25, 0.01): 178879.36557108164, (2, 1, 0, 20, 1, 25, 0.015): 12141.070542096391, (2, 1, 0, 20, 1, 25, 0.02): 1135.6170908485867, (2, 1, 0, 20, 1, 25, 0.025): 73.95168739147019, (2, 1, 0, 20, 2, 25, None): 50254342.57602336, (2, 1, 0, 20, 2, 25, 0.005): 10124226.0325845, (2, 1, 0, 20, 2, 25, 0.01): 194918.6868102128, (2, 1, 0, 20, 2, 25, 0.015): 14660.330364273506, (2, 1, 0, 20, 2, 25, 0.02): 1413.7541602542792, (2, 1, 0, 20, 2, 25, 0.025): 71.2283079934707, (2, 1, 1, 15, 1, 25, None): 4108171.090939054, (2, 1, 1, 15, 1, 25, 0.005): 608854.6572021503, (2, 1, 1, 15, 1, 25, 0.01): 2085.281179444604, (2, 1, 1, 15, 1, 25, 0.015): 1885.464530979511, (2, 1, 1, 15, 1, 25, 0.02): 1612.054938775194, (2, 1, 1, 15, 1, 25, 0.025): 1184.6502126957005, (2, 1, 1, 15, 2, 25, None): 4201350.196639275, (2, 1, 1, 15, 2, 25, 0.005): 765707.3278911278, (2, 1, 1, 15, 2, 25, 0.01): 2089.3263968856836, (2, 1, 1, 15, 2, 25, 0.015): 2337.2662365558044, (2, 1, 1, 15, 2, 25, 0.02): 2133.237069418693, (2, 1, 1, 15, 2, 25, 0.025): 1512.1045795927168, (2, 1, 1, 20, 1, 25, None): 3063824.590754712, (2, 1, 1, 20, 1, 25, 0.005): 570112.204038317, (2, 1, 1, 20, 1, 25, 0.01): 1633.3971887544224, (2, 1, 1, 20, 1, 25, 0.015): 1797.5006000072733, (2, 1, 1, 20, 1, 25, 0.02): 1607.2834257582906, (2, 1, 1, 20, 1, 25, 0.025): 1097.378794529496, (2, 1, 1, 20, 2, 25, None): 5789809.838955373, (2, 1, 1, 20, 2, 25, 0.005): 853091.1613629912, (2, 1, 1, 20, 2, 25, 0.01): 2033.4895641636701, (2, 1, 1, 20, 2, 25, 0.015): 1950.0900375593608, (2, 1, 1, 20, 2, 25, 0.02): 1890.5700962815797, (2, 1, 1, 20, 2, 25, 0.025): 1456.7088415610408, (2, 1, 2, 15, 1, 25, None): 5853270.478430453, (2, 1, 2, 15, 1, 25, 0.005): 1002347.0451098317, (2, 1, 2, 15, 1, 25, 0.01): 6581.6829271998, (2, 1, 2, 15, 1, 25, 0.015): 856.780141380831, (2, 1, 2, 15, 1, 25, 0.02): 1092.3866830177228, (2, 1, 2, 15, 1, 25, 0.025): 893.1648615913808, (2, 1, 2, 15, 2, 25, None): 9304320.155796396, (2, 1, 2, 15, 2, 25, 0.005): 1353972.3464370593, (2, 1, 2, 15, 2, 25, 0.01): 7269.512197459831, (2, 1, 2, 15, 2, 25, 0.015): 941.1240293083343, (2, 1, 2, 15, 2, 25, 0.02): 1325.675441523682, (2, 1, 2, 15, 2, 25, 0.025): 1068.8006589967172, (2, 1, 2, 20, 1, 25, None): 4053461.0218248786, (2, 1, 2, 20, 1, 25, 0.005): 1484491.7972936796, (2, 1, 2, 20, 1, 25, 0.01): 5562.696774794858, (2, 1, 2, 20, 1, 25, 0.015): 591.8963676973001, (2, 1, 2, 20, 1, 25, 0.02): 1026.7715311134339, (2, 1, 2, 20, 1, 25, 0.025): 924.2836355815472, (2, 1, 2, 20, 2, 25, None): 15794313.05296437, (2, 1, 2, 20, 2, 25, 0.005): 1427727.8977744204, (2, 1, 2, 20, 2, 25, 0.01): 5981.779528881485, (2, 1, 2, 20, 2, 25, 0.015): 644.0828393534326, (2, 1, 2, 20, 2, 25, 0.02): 1276.5291169093905, (2, 1, 2, 20, 2, 25, 0.025): 1147.9229066893383, (2, 1, 3, 15, 1, 25, None): 2491759.630787217, (2, 1, 3, 15, 1, 25, 0.005): 408630.7588859204, (2, 1, 3, 15, 1, 25, 0.01): 1963.6394233716414, (2, 1, 3, 15, 1, 25, 0.015): 2922.0949103921516, (2, 1, 3, 15, 1, 25, 0.02): 1975.3261641217657, (2, 1, 3, 15, 1, 25, 0.025): 1400.904447019137, (2, 1, 3, 15, 2, 25, None): 7077710.815742177, (2, 1, 3, 15, 2, 25, 0.005): 719944.6875691855, (2, 1, 3, 15, 2, 25, 0.01): 2288.9954601487498, (2, 1, 3, 15, 2, 25, 0.015): 3362.238128413521, (2, 1, 3, 15, 2, 25, 0.02): 2507.224787197747, (2, 1, 3, 15, 2, 25, 0.025): 1633.0932453834778, (2, 1, 3, 20, 1, 25, None): 2638295.9651146904, (2, 1, 3, 20, 1, 25, 0.005): 439750.64375898667, (2, 1, 3, 20, 1, 25, 0.01): 1347.7759532389375, (2, 1, 3, 20, 1, 25, 0.015): 2472.1924096878433, (2, 1, 3, 20, 1, 25, 0.02): 1957.2246238714083, (2, 1, 3, 20, 1, 25, 0.025): 1383.1571765721385, (2, 1, 3, 20, 2, 25, None): 3251684.628253686, (2, 1, 3, 20, 2, 25, 0.005): 505483.2615044428, (2, 1, 3, 20, 2, 25, 0.01): 1902.7865129181428, (2, 1, 3, 20, 2, 25, 0.015): 3123.809104645273, (2, 1, 3, 20, 2, 25, 0.02): 2277.0802329767153, (2, 1, 3, 20, 2, 25, 0.025): 1607.799137492837, (2, 2, 0, 15, 1, 25, None): 31603927.533332266, (2, 2, 0, 15, 1, 25, 0.005): 6569193.104145449, (2, 2, 0, 15, 1, 25, 0.01): 176789.63887660814, (2, 2, 0, 15, 1, 25, 0.015): 12064.112557513245, (2, 2, 0, 15, 1, 25, 0.02): 1000.6652733773075, (2, 2, 0, 15, 1, 25, 0.025): 45.54733223089016, (2, 2, 0, 15, 2, 25, None): 56602384.87540021, (2, 2, 0, 15, 2, 25, 0.005): 8110841.735475968, (2, 2, 0, 15, 2, 25, 0.01): 174595.90929092636, (2, 2, 0, 15, 2, 25, 0.015): 12116.10046241323, (2, 2, 0, 15, 2, 25, 0.02): 1091.7323032756897, (2, 2, 0, 15, 2, 25, 0.025): 66.28949127099264, (2, 2, 0, 20, 1, 25, None): 27355774.298336305, (2, 2, 0, 20, 1, 25, 0.005): 7383859.314721467, (2, 2, 0, 20, 1, 25, 0.01): 163889.13499079374, (2, 2, 0, 20, 1, 25, 0.015): 13960.498044332404, (2, 2, 0, 20, 1, 25, 0.02): 1304.159821604356, (2, 2, 0, 20, 1, 25, 0.025): 65.36399768003986, (2, 2, 0, 20, 2, 25, None): 61970748.4532052, (2, 2, 0, 20, 2, 25, 0.005): 8405267.181697812, (2, 2, 0, 20, 2, 25, 0.01): 203389.12007233073, (2, 2, 0, 20, 2, 25, 0.015): 16011.402899741714, (2, 2, 0, 20, 2, 25, 0.02): 1341.229880120495, (2, 2, 0, 20, 2, 25, 0.025): 79.87946239840825, (2, 2, 1, 15, 1, 25, None): 4728507.54093845, (2, 2, 1, 15, 1, 25, 0.005): 590813.5903365681, (2, 2, 1, 15, 1, 25, 0.01): 1933.0773372934786, (2, 2, 1, 15, 1, 25, 0.015): 1830.3825458289894, (2, 2, 1, 15, 1, 25, 0.02): 1676.6324611201244, (2, 2, 1, 15, 1, 25, 0.025): 1149.806171738144, (2, 2, 1, 15, 2, 25, None): 5758893.915844843, (2, 2, 1, 15, 2, 25, 0.005): 967200.7357245338, (2, 2, 1, 15, 2, 25, 0.01): 2631.1321124423052, (2, 2, 1, 15, 2, 25, 0.015): 2225.861699524964, (2, 2, 1, 15, 2, 25, 0.02): 2052.6819885128907, (2, 2, 1, 15, 2, 25, 0.025): 1384.199714306993, (2, 2, 1, 20, 1, 25, None): 8642169.660643006, (2, 2, 1, 20, 1, 25, 0.005): 668187.8013955144, (2, 2, 1, 20, 1, 25, 0.01): 1593.1498322360042, (2, 2, 1, 20, 1, 25, 0.015): 1749.4182594972665, (2, 2, 1, 20, 1, 25, 0.02): 1611.043903841805, (2, 2, 1, 20, 1, 25, 0.025): 1124.2817791779562, (2, 2, 1, 20, 2, 25, None): 6365406.196575796, (2, 2, 1, 20, 2, 25, 0.005): 1061193.6088736295, (2, 2, 1, 20, 2, 25, 0.01): 2009.2576128219018, (2, 2, 1, 20, 2, 25, 0.015): 2114.59516278776, (2, 2, 1, 20, 2, 25, 0.02): 1965.2767450933407, (2, 2, 1, 20, 2, 25, 0.025): 1520.7465371160017, (2, 2, 2, 15, 1, 25, None): 6538896.076125494, (2, 2, 2, 15, 1, 25, 0.005): 1261439.0288439763, (2, 2, 2, 15, 1, 25, 0.01): 5482.320193451273, (2, 2, 2, 15, 1, 25, 0.015): 774.5950457172404, (2, 2, 2, 15, 1, 25, 0.02): 1052.0018749224837, (2, 2, 2, 15, 1, 25, 0.025): 963.1369389947449, (2, 2, 2, 15, 2, 25, None): 9941689.582100464, (2, 2, 2, 15, 2, 25, 0.005): 1553664.1498343986, (2, 2, 2, 15, 2, 25, 0.01): 7021.881980020776, (2, 2, 2, 15, 2, 25, 0.015): 978.7125141441428, (2, 2, 2, 15, 2, 25, 0.02): 1438.8937777111526, (2, 2, 2, 15, 2, 25, 0.025): 1078.5491020361394, (2, 2, 2, 20, 1, 25, None): 5173625.647120729, (2, 2, 2, 20, 1, 25, 0.005): 889814.0762126119, (2, 2, 2, 20, 1, 25, 0.01): 6669.758244429213, (2, 2, 2, 20, 1, 25, 0.015): 591.3769596938076, (2, 2, 2, 20, 1, 25, 0.02): 945.5441723651165, (2, 2, 2, 20, 1, 25, 0.025): 921.8967875055591, (2, 2, 2, 20, 2, 25, None): 10303426.371342564, (2, 2, 2, 20, 2, 25, 0.005): 1401153.059643004, (2, 2, 2, 20, 2, 25, 0.01): 6687.597893137361, (2, 2, 2, 20, 2, 25, 0.015): 774.0758438969345, (2, 2, 2, 20, 2, 25, 0.02): 1223.258257546193, (2, 2, 2, 20, 2, 25, 0.025): 1070.1177157104255, (2, 2, 3, 15, 1, 25, None): 1641400.7583978367, (2, 2, 3, 15, 1, 25, 0.005): 403097.7892858219, (2, 2, 3, 15, 1, 25, 0.01): 1870.9381886643437, (2, 2, 3, 15, 1, 25, 0.015): 2814.0949351332943, (2, 2, 3, 15, 1, 25, 0.02): 2081.527533298886, (2, 2, 3, 15, 1, 25, 0.025): 1433.9701977888317, (2, 2, 3, 15, 2, 25, None): 4277912.935358867, (2, 2, 3, 15, 2, 25, 0.005): 526206.3322761055, (2, 2, 3, 15, 2, 25, 0.01): 2782.6676381127277, (2, 2, 3, 15, 2, 25, 0.015): 3461.218433107557, (2, 2, 3, 15, 2, 25, 0.02): 2824.188475884836, (2, 2, 3, 15, 2, 25, 0.025): 1738.248499883642, (2, 2, 3, 20, 1, 25, None): 3217547.8938439838, (2, 2, 3, 20, 1, 25, 0.005): 487431.43940014637, (2, 2, 3, 20, 1, 25, 0.01): 1508.3501136316659, (2, 2, 3, 20, 1, 25, 0.015): 2252.776924504145, (2, 2, 3, 20, 1, 25, 0.02): 1898.5448690992735, (2, 2, 3, 20, 1, 25, 0.025): 1264.7992530771353, (2, 2, 3, 20, 2, 25, None): 7641410.514147442, (2, 2, 3, 20, 2, 25, 0.005): 535011.2186851098, (2, 2, 3, 20, 2, 25, 0.01): 1962.0639313028544, (2, 2, 3, 20, 2, 25, 0.015): 3328.5468165136203, (2, 2, 3, 20, 2, 25, 0.02): 2244.49586809779, (2, 2, 3, 20, 2, 25, 0.025): 1817.3211242378345}

In [ ]:
def unique_conditions(mse):
    vals = {'manifold': set(), 'distType': set(), 'kfunc': set(), 'nn': set(), 'fitVersion': set(), 'nballs': set(), 'spacing': set()}
    conditions = ['manifold', 'distType', 'kfunc', 'nn', 'fitVersion', 'nballs', 'spacing']
    for key in mse.keys():
        for num, condition in enumerate(conditions):
            vals[condition].add(key[num])
    return vals



In [ ]:
# manifold comparison
sphere = avgmse_condition(0, -1, -1, -1, -1, -1, -1, mse1)
plane = avgmse_condition(1, -1, -1, -1, -1, -1, -1, mse1)
poincare = avgmse_condition(2, -1, -1, -1, -1, -1, -1, mse1)

# distance function comparison
isomap = avgmse_condition(-1, 0, -1, -1, -1, -1, -1, mse1)
riemannian = avgmse_condition(-1, 1, -1, -1, -1, -1, -1, mse1)
euclidean = avgmse_condition(-1, 2, -1, -1, -1, -1, -1, mse1)

# kernel function comparison
gauss = avgmse_condition(-1, -1, 0, -1, -1, -1, -1, mse1)
biweight = avgmse_condition(-1, -1, 1, -1, -1, -1, -1, mse1)
epanechnikov = avgmse_condition(-1, -1, 2, -1, -1, -1, -1, mse1)
triweight = avgmse_condition(-1, -1, 3, -1, -1, -1, -1, mse1)

# only did 20 nn for all - not sure what happened?
nn15 = avgmse_condition(-1, -1, -1, 15, -1, -1, -1, mse1)
nn20 = avgmse_condition(-1, -1, -1, 20, -1, -1, -1, mse1)

# fit version comparison
fit1 = avgmse_condition(-1, -1, -1, -1, 1, -1, -1, mse1)
fit2 = avgmse_condition(-1, -1, -1, -1, 2, -1, -1, mse1)

# only did 20 balls for all - also not sure what happened, actually used 25

# spacing comparison
rbynn = avgmse_condition(-1, -1, -1, -1, -1, -1, None, mse1)
r05 = avgmse_condition(-1, -1, -1, -1, -1, -1, 0.005, mse1)
r10 = avgmse_condition(-1, -1, -1, -1, -1, -1, 0.01, mse1)
r15 = avgmse_condition(-1, -1, -1, -1, -1, -1, 0.015, mse1)
r20 = avgmse_condition(-1, -1, -1, -1, -1, -1, 0.02, mse1)
r25 = avgmse_condition(-1, -1, -1, -1, -1, -1, 0.025, mse1)

In [ ]:
print("average error by manifold (sphere, euclidean, poincare disk): ", sphere, plane, poincare)
print("average error by distance function (isomap, true riemannian, euclidean): ", isomap, riemannian, euclidean)
print("average error by kernel function (gauss, biweight, epanechnikov, triweight): ", gauss, biweight, epanechnikov, triweight)
print("average error by number of nearest neighbors used in isomap (15, 20): ", nn15, nn20)
print("average error by analytic fit version (1, 2): ", fit1, fit2)
print("average error by spacing (nn distances, 0.005, 0.01, 0.015, 0.02, 0.025): ", rbynn, r05, r10, r15, r20, r25)

In [ ]:
def single_param(mse):
    
    vals = unique_conditions(mse)
    # manifold comparison
    print("average error by manifold: 0 = sphere, 1 = euclidean, 2 = poincare disk, 3 = torus, 4 = hyperboloid")
    for i in vals['manifold']:
        print(i, avgmse_condition(i, -1, -1, -1, -1, -1, -1, mse))

    # distance function comparison
    print("average error by distance function: 0 = estimated riemannian (isomap), 1 = true riemannian, 2 = euclidean")
    for i in vals['distType']:
        print(i, avgmse_condition(-1, i, -1, -1, -1, -1, -1, mse))

    # kernel function comparison
    print("average error by kernel function: 0 = gauss, 1 = biweight, 2 = epanechnikov, 3 = triweight")
    for i in vals['kfunc']:
        print(i, avgmse_condition(-1, -1, i, -1, -1, -1, -1, mse))

    # only did 20 nn for all - not sure what happened?
    print("average error by number of nearest neighbors used in isomap")
    for i in vals['nn']:
        print(i, avgmse_condition(-1, -1, -1, i, -1, -1, -1, mse))

    # fit version comparison
    print("average error by analytic fit equation version")
    for i in vals['fitVersion']:
        print(i, avgmse_condition(-1, -1, -1, -1, i, -1, -1, mse))

    # only did 20 balls for all - also not sure what happened, actually used 25
    print("average error by number of geodesic balls used to generate the quadratic fit")
    for i in vals['nballs']:
        print(i, avgmse_condition(-1, -1, -1, -1, -1, i, -1, mse))

    # spacing comparison
    print("average error by spacing: None = spaced by nearest neighbor distances, all other values are uniform")
    for i in vals['spacing']:
        print(i, avgmse_condition(-1, -1, -1, -1, -1, -1, i, mse))

    return None

In [ ]:
single_param(mse0)
single_param(mse1)

In [ ]:
avgmse_condition(-1, -1, -1, -1, -1, -1, -1, mse)

In [ ]:
# AAAH figure out function for getting multiple param combos
# sort tuples by error to get combos sorted from lowest to highest in output

def two_params(p1, p2, mse):
    errors = []
    vals = unique_conditions(mse)
    stuff = [-1, -1, -1, -1, -1, -1, -1]
    conditions = ['manifold', 'distType', 'kfunc', 'nn', 'fitVersion', 'nballs', 'spacing']
    param1 = conditions[p1]
    param2 = conditions[p2]
    print("average error by", param1, "and", param2)
    for i in vals[param1]:
        for j in vals[param2]:
            stuff[p1] = i
            stuff[p2] = j
            manifold, distType, kfunc, nn, fitVersion, nballs, spacing = stuff
            error = avgmse_condition(manifold, distType, kfunc, nn, fitVersion, nballs, spacing, mse)
            errors.append((i, j, error))
            print(i, j, error)
    errors.sort(key=lambda y: y[2])
    return errors
    

In [ ]:
two_params(1, 2, mse1)

two_params(3, 6, mse1)

Results!

MSE0

average error by manifold: 0 = sphere, 1 = euclidean, 2 = poincare disk, 3 = torus, 4 = hyperboloid
0 3675744094.407652
average error by distance function: 0 = estimated riemannian (isomap), 1 = true riemannian, 2 = euclidean
0 3675744094.407652
average error by kernel function: 0 = gauss, 1 = biweight, 2 = epanechnikov, 3 = triweight
0 4084671857.879307
1 3266816330.9359975
average error by number of nearest neighbors used in isomap
10 3675744094.407652
average error by analytic fit equation version
1 3675744094.407652
average error by number of geodesic balls used to generate the quadratic fit
10 78686356.36969694
5 7272801832.445606
average error by spacing: None = spaced by nearest neighbor distances, all other values are uniform
0.015 31652281.40777548
0.01 335998930.54223835
0.025 1237859.8649256157
None 15294318.459979594
0.005 21665236329.76697
0.02 5044846.404020889

Words instead of numbers:
* super duper high errors! lol this is what we get for running small point clouds with few geodesic balls
* biweight > gauss
    * by >, i mean better, not more error
* 10 > 5 
    * more geodesic balls seems to be better? well at least in the short range, i kinda want to test with more options here now
* 0.025 > 0.02 > None > 0.015 > 0.01 > 0.005
    * probably also depends on number of nearest neighbors and isomap, try in combo with those

MSE1

average error by manifold: 0 = sphere, 1 = euclidean, 2 = poincare disk, 3 = torus, 4 = hyperboloid
0 172293.42568793616
1 255515.88629544785
2 2948574.513859703
average error by distance function: 0 = estimated riemannian (isomap), 1 = true riemannian, 2 = euclidean
0 1079971.1456390554
1 1138073.662489998
2 1158339.0177140338
average error by kernel function: 0 = gauss, 1 = biweight, 2 = epanechnikov, 3 = triweight
0 3005180.6992489193
1 493535.67129220156
2 687535.022291391
3 315593.7082916045
average error by number of nearest neighbors used in isomap
20 1225283.0579963478
15 1025639.4925657102
average error by analytic fit equation version
1 950926.5986713934
2 1299995.9518906644
average error by number of geodesic balls used to generate the quadratic fit
25 1125461.275281029
average error by spacing: None = spaced by nearest neighbor distances, all other values are uniform
0.015 2069.863016992074
0.01 22718.699716429688
0.025 442.61624994184535
None 5027630.314648215
0.005 1699182.6556975956
0.02 723.5023570004666

More words
* sphere and euclidean have significantly lower average error (does the estimator do better on nonnegative curvature?)
* all three distance functions perform similarly? isomap actually has less error than the other two hmmm
* triweight > biweight > epanechnikov > gauss
* fit 1 > fit 2
* 0.025 > 0.02 > 0.015 > 0.01 > 0.005 > None

average error by distType and kfunc
0 0 2699366.9069206845
0 1 611696.2543227324
0 2 712724.2290096385
0 3 296097.1923031662
1 0 3170728.3615390686
1 1 372256.0407154171
1 2 697453.727711221
1 3 311856.519994284
2 0 3145446.829287004
2 1 496654.71883845516
2 2 652427.1101533133
2 3 338827.4125773633

average error by nn and spacing
20 0.015 2124.2119228909814
20 0.01 23494.782487184606
20 0.025 427.39332223646073
20 None 5571230.685637503
20 0.005 1753718.689307508
20 0.02 702.5853007642831
15 0.015 2015.5141110931663
15 0.01 21942.61694567477
15 0.025 457.8391776472299
15 None 4484029.9436589265
15 0.005 1644646.622087683
15 0.02 744.41941323665

* best combo: triweight and isomap? again, all three distance functions seem to perform similarly
* 20 and 15 also perform similarly, diverge most for "None" since 20 is identical to 15 but with 5 more points that maybe make the fit worse?

In [ ]:
def populate_specific(N, manifolds, distTypes, kfuncs, nns, fits, nballvals, radius, rmax = None):
    
    mses = {}
    

    for manifold in manifolds:
        for distType in distTypes:
            for kfunc in kfuncs:
                for nn in nns:
                    for fitVersion in fits:
                        if rmax == None:
                            for nballs in nballvals:
                                for spacing in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20]:
                                    mse = run_estimator(N, manifold, distType, kfunc, nn, fitVersion, nballs, spacing, radius, rmax)
                                    # print("condition: ", manifold, distType, kernels[kfunc], nn, fitVersion, nballs, spacing)
                                    # print("mse: ", mse)
                                    mses[(manifold, distType, kfunc, nn, fitVersion, nballs, spacing)] = mse
                                    print('finished spacing', spacing, 'for', nballs, 'geodesic balls')
                        else:
                            mse = run_estimator(N, manifold, distType, kfunc, nn, fitVersion, None, None, radius, rmax)
                            # print("condition: ", manifold, distType, kernels[kfunc], nn, fitVersion, nballs, spacing)
                            # print("mse: ", mse)
                            mses[(manifold, distType, kfunc, nn, fitVersion)] = mse
                            print('finished')
    return mses

In [ ]:
# Conditions: 10000 sample points from a sphere, with triweight kernel + isomap, 20 neighbors, 6 different number of balls for fit + 6 spacings
mse2 = populate_specific(10000, [0], [0], [3], [20], [1], [50, 60, 70, 80, 90, 100])

In [ ]:
print(mse2)

Results from MSE2

{(0, 0, 3, 20, 1, 50, None): 28092.562737409306, (0, 0, 3, 20, 1, 50, 0.005): 877.3265286562385, (0, 0, 3, 20, 1, 50, 0.01): 172.1209127943344, (0, 0, 3, 20, 1, 50, 0.015): 42.92555442055094, (0, 0, 3, 20, 1, 50, 0.02): 15.454128890094982, (0, 0, 3, 20, 1, 50, 0.025): 6.484403427264891, (0, 0, 3, 20, 1, 60, None): 11053.463137103638, (0, 0, 3, 20, 1, 60, 0.005): 598.2343592894199, (0, 0, 3, 20, 1, 60, 0.01): 100.29227109587367, (0, 0, 3, 20, 1, 60, 0.015): 19.226594601723647, (0, 0, 3, 20, 1, 60, 0.02): 6.268514258243956, (0, 0, 3, 20, 1, 60, 0.025): 2.585050343379976, (0, 0, 3, 20, 1, 70, None): 5805.923913300942, (0, 0, 3, 20, 1, 70, 0.005): 393.5333833214771, (0, 0, 3, 20, 1, 70, 0.01): 50.46990799575658, (0, 0, 3, 20, 1, 70, 0.015): 9.983274390442638, (0, 0, 3, 20, 1, 70, 0.02): 3.0872973504878427, (0, 0, 3, 20, 1, 70, 0.025): 1.0529051468771544, (0, 0, 3, 20, 1, 80, None): 2987.6881628608935, (0, 0, 3, 20, 1, 80, 0.005): 243.74879956431803, (0, 0, 3, 20, 1, 80, 0.01): 30.18905377062481, (0, 0, 3, 20, 1, 80, 0.015): 5.159702058667669, (0, 0, 3, 20, 1, 80, 0.02): 1.478197059449309, (0, 0, 3, 20, 1, 80, 0.025): 0.4955512821566918, (0, 0, 3, 20, 1, 90, None): 2230.736669729958, (0, 0, 3, 20, 1, 90, 0.005): 141.846021974195, (0, 0, 3, 20, 1, 90, 0.01): 12.85574794706257, (0, 0, 3, 20, 1, 90, 0.015): 2.7298984658982963, (0, 0, 3, 20, 1, 90, 0.02): 0.9710844939857322, (0, 0, 3, 20, 1, 90, 0.025): 0.27619918426959555, (0, 0, 3, 20, 1, 100, None): 1468.7708680160943, (0, 0, 3, 20, 1, 100, 0.005): 112.14830122973977, (0, 0, 3, 20, 1, 100, 0.01): 9.230499814420243, (0, 0, 3, 20, 1, 100, 0.015): 2.0532733505035186, (0, 0, 3, 20, 1, 100, 0.02): 0.47137161833066105, (0, 0, 3, 20, 1, 100, 0.025): 0.16464779100694707}



In [ ]:
print(unique_conditions(mse2))
single_param(mse2)
two_params(5, 6, mse2)

Taking radii by the next neighbor consistently performs the worst -- is uniform radii spacing consistent with what we proved in the overleaf?
Best performer: 100 with 0.025 spacing, check how many data points previous experiments were using for the quadratic fit model and their approximate spacing -- at what radius does the error start going up again?

average error by number of geodesic balls used to generate the quadratic fit
100 265.47316030334923
70 1044.0084469176638
80 544.7932444326851
50 4867.812377599631
90 398.23593696589484
60 1963.344987782046
average error by spacing: None = spaced by nearest neighbor distances, all other values are uniform
0.015 13.679716214631119
0.01 62.526398903012044
0.025 1.843126195825876
None 8606.524248070138
0.005 394.4728990058981
0.02 4.621765611765414

average error by nballs and spacing
100 0.015 2.0532733505035186
100 0.01 9.230499814420243
100 0.025 0.16464779100694707
100 None 1468.7708680160943
100 0.005 112.14830122973977
100 0.02 0.47137161833066105
70 0.015 9.983274390442638
70 0.01 50.46990799575658
70 0.025 1.0529051468771544
70 None 5805.923913300942
70 0.005 393.5333833214771
70 0.02 3.0872973504878427
80 0.015 5.159702058667669
80 0.01 30.18905377062481
80 0.025 0.4955512821566918
80 None 2987.6881628608935
80 0.005 243.74879956431803
80 0.02 1.478197059449309
50 0.015 42.92555442055094
50 0.01 172.1209127943344
50 0.025 6.484403427264891
50 None 28092.562737409306
50 0.005 877.3265286562385
50 0.02 15.454128890094982
90 0.015 2.7298984658982963
90 0.01 12.85574794706257
90 0.025 0.27619918426959555
90 None 2230.736669729958
90 0.005 141.846021974195
90 0.02 0.9710844939857322
60 0.015 19.226594601723647
60 0.01 100.29227109587367
60 0.025 2.585050343379976
60 None 11053.463137103638
60 0.005 598.2343592894199
60 0.02 6.268514258243956

6/29/22: What's Up
* turns out the distance functions were all performing similarly because... I trolled and never passed in unique distance functions to the estimator
* uniform radii spacing SEEMS to be better but also does the scale of the radii matter? I'm gonna try running it with larger spacings and still "None" to compare
* look into analytic fit equation a little bit tomorrow maybe

In [ ]:

mse3 = populate_specific(1000, [0], [0, 1, 2], [3], [20], [1], [25, 50]) # None, 0.025, 0.05, 0.075, 0.1

In [ ]:
mse4 = populate_specific(5000, [0], [0, 1, 2], [3], [20], [1], [25, 50]) # None, 0.025, 0.05, 0.075, 0.1

Results from MSE3

{(0, 0, 3, 20, 1, 25, None): 1920.0813489425493, (0, 0, 3, 20, 1, 25, 0.025): 99.6986925157867, (0, 0, 3, 20, 1, 25, 0.05): 15.343763183701888, (0, 0, 3, 20, 1, 25, 0.075): 2.2890005385245695, (0, 0, 3, 20, 1, 25, 0.1): 0.4450771630927793, (0, 0, 3, 20, 1, 50, None): 109.32139887144963, (0, 0, 3, 20, 1, 50, 0.025): 7.0934901981684595, (0, 0, 3, 20, 1, 50, 0.05): 0.4399303760837747, (0, 0, 3, 20, 1, 50, 0.075): 0.29169492852414763, (0, 0, 3, 20, 1, 50, 0.1): 0.8321705360003601, (0, 1, 3, 20, 1, 25, None): 1103.6438116709237, (0, 1, 3, 20, 1, 25, 0.025): 80.81578061719041, (0, 1, 3, 20, 1, 25, 0.05): 11.829912523223026, (0, 1, 3, 20, 1, 25, 0.075): 3.0074941698623996, (0, 1, 3, 20, 1, 25, 0.1): 0.8680590780687172, (0, 1, 3, 20, 1, 50, None): 77.33706036467169, (0, 1, 3, 20, 1, 50, 0.025): 4.199278707812423, (0, 1, 3, 20, 1, 50, 0.05): 0.3008711391533025, (0, 1, 3, 20, 1, 50, 0.075): 0.2797923124474328, (0, 1, 3, 20, 1, 50, 0.1): 0.8302735578504097, (0, 2, 3, 20, 1, 25, None): 2143.6092745200544, (0, 2, 3, 20, 1, 25, 0.025): 58.39875358876957, (0, 2, 3, 20, 1, 25, 0.05): 13.005836775179901, (0, 2, 3, 20, 1, 25, 0.075): 4.650060505338316, (0, 2, 3, 20, 1, 25, 0.1): 1.964468644858455, (0, 2, 3, 20, 1, 50, None): 113.37353931686992, (0, 2, 3, 20, 1, 50, 0.025): 8.760753909751994, (0, 2, 3, 20, 1, 50, 0.05): 1.6541958069901752, (0, 2, 3, 20, 1, 50, 0.075): 0.49820913809056927, (0, 2, 3, 20, 1, 50, 0.1): 0.8955140010643583}

average error by distance function: 0 = estimated riemannian (isomap), 1 = true riemannian, 2 = euclidean
0 215.58365672538815
1 128.31123341412035
2 234.68106062069677
average error by number of geodesic balls used to generate the quadratic fit
25 363.9767556291416
50 21.74054487766191
average error by spacing: None = spaced by nearest neighbor distances, all other values are uniform
0.1 0.9725938301558464
0.075 1.836041932131239
0.05 7.095751634055344
0.025 43.16112492291327
None 911.2277389477531

Results from MSE4

{(0, 0, 3, 20, 1, 25, None): 110222.50229643128, (0, 0, 3, 20, 1, 25, 0.025): 150.05048687376095, (0, 0, 3, 20, 1, 25, 0.05): 9.21558509617658, (0, 0, 3, 20, 1, 25, 0.075): 1.472810761030568, (0, 0, 3, 20, 1, 25, 0.1): 0.4850615840607441, (0, 0, 3, 20, 1, 50, None): 4189.8689788566835, (0, 0, 3, 20, 1, 50, 0.025): 6.563880234488424, (0, 0, 3, 20, 1, 50, 0.05): 0.27251016589205157, (0, 0, 3, 20, 1, 50, 0.075): 0.30509346184560565, (0, 0, 3, 20, 1, 50, 0.1): 0.8256638217469386, (0, 1, 3, 20, 1, 25, None): 109559.60444862841, (0, 1, 3, 20, 1, 25, 0.025): 143.71172349054626, (0, 1, 3, 20, 1, 25, 0.05): 9.979275885140973, (0, 1, 3, 20, 1, 25, 0.075): 1.8711224346444175, (0, 1, 3, 20, 1, 25, 0.1): 0.38993605182320534, (0, 1, 3, 20, 1, 50, None): 4330.5380310446035, (0, 1, 3, 20, 1, 50, 0.025): 5.706222177992288, (0, 1, 3, 20, 1, 50, 0.05): 0.27949841458691915, (0, 1, 3, 20, 1, 50, 0.075): 0.31161474880236323, (0, 1, 3, 20, 1, 50, 0.1): 0.8296297741772835, (0, 2, 3, 20, 1, 25, None): 137328.2468356095, (0, 2, 3, 20, 1, 25, 0.025): 115.10022908205269, (0, 2, 3, 20, 1, 25, 0.05): 13.91483359993351, (0, 2, 3, 20, 1, 25, 0.075): 5.331541839544773, (0, 2, 3, 20, 1, 25, 0.1): 2.0918809122961988, (0, 2, 3, 20, 1, 50, None): 5039.479608293518, (0, 2, 3, 20, 1, 50, 0.025): 10.347811138736605, (0, 2, 3, 20, 1, 50, 0.05): 1.8996976717114906, (0, 2, 3, 20, 1, 50, 0.075): 0.503431134950695, (0, 2, 3, 20, 1, 50, 0.1): 0.8976561620183958}

average error by distance function: 0 = estimated riemannian (isomap), 1 = true riemannian, 2 = euclidean
0 11458.156236728697
1 11405.322150265074
2 14251.781352544425
average error by number of geodesic balls used to generate the quadratic fit
25 23837.59787121868
50 905.9086218067835
average error by spacing: None = spaced by nearest neighbor distances, all other values are uniform
0.1 0.9199713843537943
0.075 1.6326023968030705
0.05 5.9269001389069205
0.025 71.91339216626288
None 61778.373366477324

* As N increases, estimated Riemannian is closer to true Riemannian, better than Euclidean
* Increasing the number of geodesic balls and radii both seem to help -- until when? Rerun on smaller data set (similar to MSE3)
* rmax calculations:
    * 1000 sample points
        * 

In [ ]:
single_param(mse3)
single_param(mse4)

Results from MSE5-7 (dictionaries collapsed)

average error by distance function: 0 = estimated riemannian (isomap), 1 = true riemannian, 2 = euclidean
0 368.79586501340225
1 540.8834025115852
2 503.358058613757
average error by kernel function: 0 = gauss, 1 = biweight, 2 = epanechnikov, 3 = triweight
0 1193.584007970912
1 229.85692165110018
2 320.91686319755786
3 139.69197536542308
average error by number of geodesic balls used to generate the quadratic fit
25 898.2711115387169
50 43.75377255377943
average error by spacing: None = spaced by nearest neighbor distances, all other values are uniform
0.05 5.146127951763173
0.1 0.8640640161427143
0.2 1.7510802543082757
0.15 1.1668599912920132
None 2346.1340780177347

average error by distType and kfunc
0 0 853.277648621173
0 1 221.53559340407446
0 2 277.5558841274243
0 3 122.81433390093703
1 0 1499.9980439474173
1 1 163.50998348024922
1 2 356.72606014600854
1 3 143.29952247266587
2 0 1227.4763313441442
2 1 304.525188068977
2 2 328.4686453192404
2 3 152.9620697226663
[(0, 3, 122.81433390093703), (1, 3, 143.29952247266587), (2, 3, 152.9620697226663), (1, 1, 163.50998348024922), (0, 1, 221.53559340407446), (0, 2, 277.5558841274243), (2, 1, 304.525188068977), (2, 2, 328.4686453192404), (1, 2, 356.72606014600854), (0, 0, 853.277648621173), (2, 0, 1227.4763313441442), (1, 0, 1499.9980439474173)]
average error by nballs and spacing
25 0.05 9.489657315516522
25 0.1 0.8671630145038604
25 0.2 0.8231628756553656
25 0.15 0.3620409980653289
25 None 4479.813533489844
50 0.05 0.8025985880098232
50 0.1 0.8609650177815683
50 0.2 2.678997632961186
50 0.15 1.9716789845186975
50 None 212.45462254562582
[(25, 0.15, 0.3620409980653289), (50, 0.05, 0.8025985880098232), (25, 0.2, 0.8231628756553656), (50, 0.1, 0.8609650177815683), (25, 0.1, 0.8671630145038604), (50, 0.15, 1.9716789845186975), (50, 0.2, 2.678997632961186), (25, 0.05, 9.489657315516522), (50, None, 212.45462254562582), (25, None, 4479.813533489844)]

* Triweight kernel is consistently best regardless of the distance function, isomap has lowest overall error (estimated riemannian + triweight)
* error decreases as number of geodesic balls is increased -- still need to experiment with until when
* nballs and spacing -- huh they seem to be somewhat dependent?
    * for just 25, None < 0.05 < 0.1 < 0.15 > 0.2 -- rmax of 25*0.15 = 3.75
    * for 50, None < 0.05 > 0.1 > 0.15 > 0.2 -- rmax of 50*0.05 = 2.5
    * There's definitely some inversely proportional relationship... I'm going to run MSE8 as MSE7 with even more incremented values since runtimes are ok

MSE5

{(0, 0, 0, 20, 1, 25, None): 8055.507344568977, (0, 0, 0, 20, 1, 25, 0.05): 1.614593404300184, (0, 0, 0, 20, 1, 25, 0.1): 0.14170065201170304, (0, 0, 0, 20, 1, 25, 0.15): 0.2948799262429387, (0, 0, 0, 20, 1, 25, 0.2): 0.8253326237766232, (0, 0, 0, 20, 1, 50, None): 468.75012037200315, (0, 0, 0, 20, 1, 50, 0.05): 0.1423818229144714, (0, 0, 0, 20, 1, 50, 0.1): 0.8477171196486079, (0, 0, 0, 20, 1, 50, 0.15): 1.9725229427646511, (0, 0, 0, 20, 1, 50, 0.2): 2.6798927790917126, (0, 0, 1, 20, 1, 25, None): 2133.4739408085106, (0, 0, 1, 20, 1, 25, 0.05): 11.368738326524156, (0, 0, 1, 20, 1, 25, 0.1): 0.591794143813506, (0, 0, 1, 20, 1, 25, 0.15): 0.2862628453229236, (0, 0, 1, 20, 1, 25, 0.2): 0.7907429440515373, (0, 0, 1, 20, 1, 50, None): 63.07432841241818, (0, 0, 1, 20, 1, 50, 0.05): 0.30035435903330543, (0, 0, 1, 20, 1, 50, 0.1): 0.8282310921480899, (0, 0, 1, 20, 1, 50, 0.15): 1.9653455056090834, (0, 0, 1, 20, 1, 50, 0.2): 2.676195603313269, (0, 0, 2, 20, 1, 25, None): 2663.563055908067, (0, 0, 2, 20, 1, 25, 0.05): 11.244977629517432, (0, 0, 2, 20, 1, 25, 0.1): 0.45743120212365707, (0, 0, 2, 20, 1, 25, 0.15): 0.2984011627202675, (0, 0, 2, 20, 1, 25, 0.2): 0.7994143026281816, (0, 0, 2, 20, 1, 50, None): 93.30064738912351, (0, 0, 2, 20, 1, 50, 0.05): 0.42662925287317444, (0, 0, 2, 20, 1, 50, 0.1): 0.8262896712395406, (0, 0, 2, 20, 1, 50, 0.15): 1.9642194294963085, (0, 0, 2, 20, 1, 50, 0.2): 2.6777753264542627, (0, 0, 3, 20, 1, 25, None): 1125.876616075166, (0, 0, 3, 20, 1, 25, 0.05): 11.838640144752597, (0, 0, 3, 20, 1, 25, 0.1): 0.3751973651777272, (0, 0, 3, 20, 1, 25, 0.15): 0.31046476139535545, (0, 0, 3, 20, 1, 25, 0.2): 0.7894073093125521, (0, 0, 3, 20, 1, 50, None): 83.19635640035611, (0, 0, 3, 20, 1, 50, 0.05): 0.2924061325769845, (0, 0, 3, 20, 1, 50, 0.1): 0.8240356820129532, (0, 0, 3, 20, 1, 50, 0.15): 1.964503416289817, (0, 0, 3, 20, 1, 50, 0.2): 2.675711722330189, (0, 1, 0, 20, 1, 25, None): 14429.09436283156, (0, 1, 0, 20, 1, 25, 0.05): 1.0555348260986643, (0, 1, 0, 20, 1, 25, 0.1): 0.13382049933153412, (0, 1, 0, 20, 1, 25, 0.15): 0.31476857130035907, (0, 1, 0, 20, 1, 25, 0.2): 0.8271629365663176, (0, 1, 0, 20, 1, 50, None): 562.8869503644147, (0, 1, 0, 20, 1, 50, 0.05): 0.16633999080867204, (0, 1, 0, 20, 1, 50, 0.1): 0.8486346078254998, (0, 1, 0, 20, 1, 50, 0.15): 1.9728527652674825, (0, 1, 0, 20, 1, 50, 0.2): 2.680012081001021, (0, 1, 1, 20, 1, 25, None): 1538.3325310787843, (0, 1, 1, 20, 1, 25, 0.05): 10.93108308506438, (0, 1, 1, 20, 1, 25, 0.1): 0.5201093115779524, (0, 1, 1, 20, 1, 25, 0.15): 0.3089379430689144, (0, 1, 1, 20, 1, 25, 0.2): 0.8019594532239934, (0, 1, 1, 20, 1, 50, None): 78.223665444294, (0, 1, 1, 20, 1, 50, 0.05): 0.48118557403547013, (0, 1, 1, 20, 1, 50, 0.1): 0.8527013575559631, (0, 1, 1, 20, 1, 50, 0.15): 1.9686259249982345, (0, 1, 1, 20, 1, 50, 0.2): 2.6790356298892624, (0, 1, 2, 20, 1, 25, None): 3450.9122686828646, (0, 1, 2, 20, 1, 25, 0.05): 7.562322403348044, (0, 1, 2, 20, 1, 25, 0.1): 0.3298057067348213, (0, 1, 2, 20, 1, 25, 0.15): 0.29658650869567044, (0, 1, 2, 20, 1, 25, 0.2): 0.7975752340605706, (0, 1, 2, 20, 1, 50, None): 101.52483782508067, (0, 1, 2, 20, 1, 50, 0.05): 0.35022177207202043, (0, 1, 2, 20, 1, 50, 0.1): 0.8382475591593244, (0, 1, 2, 20, 1, 50, 0.15): 1.9712715866851238, (0, 1, 2, 20, 1, 50, 0.2): 2.677464181385234, (0, 1, 3, 20, 1, 25, None): 1328.3917916473752, (0, 1, 3, 20, 1, 25, 0.05): 13.893055933226462, (0, 1, 3, 20, 1, 25, 0.1): 0.5953606338979824, (0, 1, 3, 20, 1, 25, 0.15): 0.28251073300086843, (0, 1, 3, 20, 1, 25, 0.2): 0.7655305674620851, (0, 1, 3, 20, 1, 50, None): 83.34636741570684, (0, 1, 3, 20, 1, 50, 0.05): 0.25759547109190845, (0, 1, 3, 20, 1, 50, 0.1): 0.8341710570204063, (0, 1, 3, 20, 1, 50, 0.15): 1.9540269978314877, (0, 1, 3, 20, 1, 50, 0.2): 2.6748142700456374, (0, 2, 0, 20, 1, 25, None): 11550.821587835171, (0, 2, 0, 20, 1, 25, 0.05): 4.801935093406424, (0, 2, 0, 20, 1, 25, 0.1): 1.7946162538024362, (0, 2, 0, 20, 1, 25, 0.15): 0.5041118339809277, (0, 2, 0, 20, 1, 25, 0.2): 0.8994918165575586, (0, 2, 0, 20, 1, 50, None): 708.6789288981643, (0, 2, 0, 20, 1, 50, 0.05): 1.6695150556593485, (0, 2, 0, 20, 1, 50, 0.1): 0.9234476007618639, (0, 2, 0, 20, 1, 50, 0.15): 1.9864492372703035, (0, 2, 0, 20, 1, 50, 0.2): 2.6832298166672217, (0, 2, 1, 20, 1, 25, None): 2939.6435245249636, (0, 2, 1, 20, 1, 25, 0.05): 13.47483711199411, (0, 2, 1, 20, 1, 25, 0.1): 1.6602539766353879, (0, 2, 1, 20, 1, 25, 0.15): 0.4777117985905527, (0, 2, 1, 20, 1, 25, 0.2): 0.8634366794856062, (0, 2, 1, 20, 1, 50, None): 81.77841402933106, (0, 2, 1, 20, 1, 50, 0.05): 1.7888081093627852, (0, 2, 1, 20, 1, 50, 0.1): 0.8975197972092102, (0, 2, 1, 20, 1, 50, 0.15): 1.9854203649850029, (0, 2, 1, 20, 1, 50, 0.2): 2.6819542972125947, (0, 2, 2, 20, 1, 25, None): 3129.988448238336, (0, 2, 2, 20, 1, 25, 0.05): 10.528212011304321, (0, 2, 2, 20, 1, 25, 0.1): 1.9663078212910294, (0, 2, 2, 20, 1, 25, 0.15): 0.500392743947454, (0, 2, 2, 20, 1, 25, 0.2): 0.8567856990986297, (0, 2, 2, 20, 1, 50, None): 133.2528784329672, (0, 2, 2, 20, 1, 50, 0.05): 2.0234458993892797, (0, 2, 2, 20, 1, 50, 0.1): 0.9082873467396191, (0, 2, 2, 20, 1, 50, 0.15): 1.9798720940902568, (0, 2, 2, 20, 1, 50, 0.2): 2.6818229052401557, (0, 2, 3, 20, 1, 25, None): 1412.1569296783457, (0, 2, 3, 20, 1, 25, 0.2): 0.8611149416407319, (0, 2, 3, 20, 1, 50, None): 91.44197556365057, (0, 2, 3, 20, 1, 50, 0.05): 1.7322996163004596, (0, 2, 3, 20, 1, 50, 0.1): 0.9022973220577392, (0, 2, 3, 20, 1, 50, 0.15): 1.975037548936616, (0, 2, 3, 20, 1, 50, 0.2): 2.680062982903668}

MSE6

{(0, 0, 3, 20, 1, 20, None): 4004.0518577100793, (0, 0, 3, 20, 1, 20, 0.05): 29.941320801898662, (0, 0, 3, 20, 1, 20, 0.1): 1.8800603163062708, (0, 0, 3, 20, 1, 20, 0.15): 0.24353811186233054, (0, 0, 3, 20, 1, 20, 0.2): 0.3369022735822524, (0, 0, 3, 20, 1, 30, None): 499.9985915644881, (0, 0, 3, 20, 1, 30, 0.05): 5.521631996156251, (0, 0, 3, 20, 1, 30, 0.1): 0.2613592123252546, (0, 0, 3, 20, 1, 30, 0.15): 0.5598815298983165, (0, 0, 3, 20, 1, 30, 0.2): 1.2895758413102059, (0, 0, 3, 20, 1, 40, None): 193.26144153190273, (0, 0, 3, 20, 1, 40, 0.05): 1.076423339040104, (0, 0, 3, 20, 1, 40, 0.1): 0.36117780645288666, (0, 0, 3, 20, 1, 40, 0.15): 1.308584448850505, (0, 0, 3, 20, 1, 40, 0.2): 2.127790911863222, (0, 0, 3, 20, 1, 50, None): 95.26632132528455, (0, 0, 3, 20, 1, 50, 0.05): 0.25801221659428697, (0, 0, 3, 20, 1, 50, 0.1): 0.8300180391304478, (0, 0, 3, 20, 1, 50, 0.15): 1.9672090597642955, (0, 0, 3, 20, 1, 50, 0.2): 2.6761639339963166}

MSE7

{(0, 0, 3, 20, 1, 20, None): 28388.907570171334, (0, 0, 3, 20, 1, 20, 0.05): 38.15911410040456, (0, 0, 3, 20, 1, 20, 0.1): 1.8031246426083543, (0, 0, 3, 20, 1, 20, 0.15): 0.26388310986639346, (0, 0, 3, 20, 1, 20, 0.2): 0.34780737221825997, (0, 0, 3, 20, 1, 30, None): 3627.5111544557303, (0, 0, 3, 20, 1, 30, 0.05): 5.506256240942105, (0, 0, 3, 20, 1, 30, 0.1): 0.2596939379717755, (0, 0, 3, 20, 1, 30, 0.15): 0.5578448328308, (0, 0, 3, 20, 1, 30, 0.2): 1.2904282877018776, (0, 0, 3, 20, 1, 40, None): 849.0186240748856, (0, 0, 3, 20, 1, 40, 0.05): 0.8719677059049049, (0, 0, 3, 20, 1, 40, 0.1): 0.3781874401388547, (0, 0, 3, 20, 1, 40, 0.15): 1.3096683000530032, (0, 0, 3, 20, 1, 40, 0.2): 2.1285229218430852, (0, 0, 3, 20, 1, 50, None): 386.2802854721445, (0, 0, 3, 20, 1, 50, 0.05): 0.29216511708232495, (0, 0, 3, 20, 1, 50, 0.1): 0.8216513008134044, (0, 0, 3, 20, 1, 50, 0.15): 1.9591489458008196, (0, 0, 3, 20, 1, 50, 0.2): 2.673195839389473}

In [ ]:
mse5 = populate_specific(1000, [0], [0, 1, 2], [0, 1, 2, 3], [20], [1], [25, 50]) # None, 0.05, 0.1, 0.15, 0.2

In [ ]:
single_param(mse5)
print(two_params(1, 2, mse5))
print(two_params(5, 6, mse5))

In [ ]:
mse6 = populate_specific(1000, [0], [0], [3], [20], [1], [20, 30, 40, 50])
mse7 = populate_specific(2000, [0], [0], [3], [20], [1], [20, 30, 40, 50])

In [ ]:
mse8 = populate_specific(1000, [0], [0], [3], [20], [1], [20, 25, 30, 35, 40, 45, 50])
mse9 = populate_specific(2000, [0], [0], [3], [20], [1], [20, 25, 30, 35, 40, 45, 50])

MSE8

{(0, 0, 3, 20, 1, 20, 0.05): 34.770850277632825, (0, 0, 3, 20, 1, 20, 0.075): 7.677609234894786, (0, 0, 3, 20, 1, 20, 0.1): 2.1150558941454527, (0, 0, 3, 20, 1, 20, 0.125): 0.6541512806250044, (0, 0, 3, 20, 1, 20, 0.15): 0.30986155099473595, (0, 0, 3, 20, 1, 25, 0.05): 11.591793723892573, (0, 0, 3, 20, 1, 25, 0.075): 2.1129223845223812, (0, 0, 3, 20, 1, 25, 0.1): 0.5788196996220024, (0, 0, 3, 20, 1, 25, 0.125): 0.22012399422456128, (0, 0, 3, 20, 1, 25, 0.15): 0.2794969098781876, (0, 0, 3, 20, 1, 30, 0.05): 4.719588443237982, (0, 0, 3, 20, 1, 30, 0.075): 1.1086073482333518, (0, 0, 3, 20, 1, 30, 0.1): 0.2719411401488417, (0, 0, 3, 20, 1, 30, 0.125): 0.26228155293292854, (0, 0, 3, 20, 1, 30, 0.15): 0.5795239524134337, (0, 0, 3, 20, 1, 35, 0.05): 2.064562877454522, (0, 0, 3, 20, 1, 35, 0.075): 0.352944846533811, (0, 0, 3, 20, 1, 35, 0.1): 0.26444712010154114, (0, 0, 3, 20, 1, 35, 0.125): 0.5072687307414411, (0, 0, 3, 20, 1, 35, 0.15): 0.9342655377061229, (0, 0, 3, 20, 1, 40, 0.05): 1.37697321266912, (0, 0, 3, 20, 1, 40, 0.075): 0.20851944566520245, (0, 0, 3, 20, 1, 40, 0.1): 0.35316476600305546, (0, 0, 3, 20, 1, 40, 0.125): 0.8214723980309157, (0, 0, 3, 20, 1, 40, 0.15): 1.295601363191337, (0, 0, 3, 20, 1, 45, 0.05): 0.6114655503379957, (0, 0, 3, 20, 1, 45, 0.075): 0.22551539048951666, (0, 0, 3, 20, 1, 45, 0.1): 0.5638755808413292, (0, 0, 3, 20, 1, 45, 0.125): 1.132703906215349, (0, 0, 3, 20, 1, 45, 0.15): 1.6603090841392754, (0, 0, 3, 20, 1, 50, 0.05): 0.2781871117591686, (0, 0, 3, 20, 1, 50, 0.075): 0.2702076611887964, (0, 0, 3, 20, 1, 50, 0.1): 0.8215378259923559, (0, 0, 3, 20, 1, 50, 0.125): 1.4399332544166314, (0, 0, 3, 20, 1, 50, 0.15): 1.9641762631817274}

MSE 9

{(0, 0, 3, 20, 1, 20, 0.05): 37.50092273038508, (0, 0, 3, 20, 1, 20, 0.075): 7.849600819046602, (0, 0, 3, 20, 1, 20, 0.1): 1.984705085542726, (0, 0, 3, 20, 1, 20, 0.125): 0.4951883826407261, (0, 0, 3, 20, 1, 20, 0.15): 0.24075875862970844, (0, 0, 3, 20, 1, 25, 0.05): 12.002512455199618, (0, 0, 3, 20, 1, 25, 0.075): 2.197791003760717, (0, 0, 3, 20, 1, 25, 0.1): 0.5322517543641535, (0, 0, 3, 20, 1, 25, 0.125): 0.18499938809511599, (0, 0, 3, 20, 1, 25, 0.15): 0.29018921200687053, (0, 0, 3, 20, 1, 30, 0.05): 4.619805015563386, (0, 0, 3, 20, 1, 30, 0.075): 0.7059805811834975, (0, 0, 3, 20, 1, 30, 0.1): 0.22273704826067894, (0, 0, 3, 20, 1, 30, 0.125): 0.29495636208752507, (0, 0, 3, 20, 1, 30, 0.15): 0.5626579024693724, (0, 0, 3, 20, 1, 35, 0.05): 2.7635532829063485, (0, 0, 3, 20, 1, 35, 0.075): 0.30479088064510323, (0, 0, 3, 20, 1, 35, 0.1): 0.2359831252884561, (0, 0, 3, 20, 1, 35, 0.125): 0.5191571247368408, (0, 0, 3, 20, 1, 35, 0.15): 0.9286481593287164, (0, 0, 3, 20, 1, 40, 0.05): 0.9997345960054317, (0, 0, 3, 20, 1, 40, 0.075): 0.22443407271552712, (0, 0, 3, 20, 1, 40, 0.1): 0.36167738124980103, (0, 0, 3, 20, 1, 40, 0.125): 0.8132932130343536, (0, 0, 3, 20, 1, 40, 0.15): 1.3090778587591414, (0, 0, 3, 20, 1, 45, 0.05): 0.7440712299363306, (0, 0, 3, 20, 1, 45, 0.075): 0.2193971818696302, (0, 0, 3, 20, 1, 45, 0.1): 0.5794544768139153, (0, 0, 3, 20, 1, 45, 0.125): 1.1371071390583884, (0, 0, 3, 20, 1, 45, 0.15): 1.6582791069226246, (0, 0, 3, 20, 1, 50, 0.05): 0.38548399022933155, (0, 0, 3, 20, 1, 50, 0.075): 0.28815299182474075, (0, 0, 3, 20, 1, 50, 0.1): 0.8253083169538554, (0, 0, 3, 20, 1, 50, 0.125): 1.4405289353047783, (0, 0, 3, 20, 1, 50, 0.15): 1.960354850640431}

In [ ]:
single_param(mse6)
print(two_params(5, 6, mse6))
single_param(mse7)
print(two_params(5, 6, mse7))

In [ ]:
print(mse8)
print(mse9)

6/29/22: Compilation of N vs. nballs vs. spacing from MSE1-9
Constant conditions:
* manifold: 0 (sphere)
* distType: 0 (estimated Riemannian)
* kfunc: 3 for mses1-9
* nn: 20
* fitVersion: 1
* sphere radius = 1

MSE1 - 1000 sample points

* 25 * 0.025 = 0.625

MSE2 - 10000 sample points

* 100 * 0.025 = 2.5
* 90 * 0.025 = 2.25
* 80 * 0.025 = 2
* 70 * 0.025 = 1.75
* 60 * 0.025 = 1.5
* 50 * 0.025 = 1.25

MSE3 - 1000 sample points

* 50 * 0.075 = 3.75
* 25 * 0.1 = 2.5

MSE4 - 5000 sample points

* 50 * 0.05 = 2.5
* 25 * 0.1 = 2.5

* 50 * 0.05 = 2.5
* 50 * 0.075 = 3.75

MSE5 - 1000 sample points

* 50 * 0.05 = 2.5
* 25 * 0.15 = 3.75

* 50 * 0.05 = 2.5
* 25 * 0.15 = 3.75

MSE6 - 1000 sample points

* 50 * 0.075 = 3.75
* 40 * 0.075 = 3
* 30 * 0.1 = 3
* 20 * 0.15 = 3

* 40 * 0.075 = 3
* 20 * 0.15 = 3
* 30 * 0.1 = 3
* 30 * 0.125 = 3.75
* 50 * 0.075 = 3.75

MSE7 - 2000 sample points

* 50 * 0.075 = 3.75
* 40 * 0.075 = 3
* 30 * 0.1 = 3
* 20 * 0.15 = 3

* 40 * 0.075 = 3
* 20 * 0.15 = 3
* 30 * 0.1 = 3
* 30 * 0.125 = 3.75
* 50 * 0.075 = 3.75

MSE8 - 1000 sample points
* 50 * 0.075 = 3.75
* 45 * 0.075 = 3.375
* 40 * 0.075 = 3
* 35 * 0.1 = 3.5
* 30 * 0.125 = 3.75
* 25 * 0.125 = 3.125
* 20 * 0.15 = 3

* 40 * 0.075 = 3
* 25 * 0.125 = 3.125
* 45 * 0.075 = 3.375
* 30 * 0.125 = 3.75
* 35 * 0.1 = 3.5
* 50 * 0.075 = 3.75
* 30 * 0.1 = 3

MSE9 - 2000 sample points

* 50 * 0.075 = 3.75
* 45 * 0.075 = 3.375
* 40 * 0.075 = 3
* 35 * 0.1 = 3.5
* 30 * 0.1 = 3
* 25 * 0.125 = 3.125
* 20 * 0.15 = 3

* 25 * 0.125 = 3.125
* 45 * 0.075 = 3.375
* 30 * 0.1 = 3
* 40 * 0.075 = 3
* 35 * 0.1 = 3.5
* 20 * 0.15 = 3
* 50 * 0.075 = 3.75

Thoughts:
* doesn't really depend on N (no differences between 1000 and 2000 sample point experiments)
* what does it mean for the rmax covered to be greater than the radius of the sphere? OH WAIT for a sphere of radius 2, a hemisphere is approx radius pi = 3.14ish?? what about on other manifolds?

In [ ]:
def adapt_mse(manifold = -1, distType = -1, kfunc = -1, nn = -1, fitVersion = -1, nballs = -1, spacing = -1, mses = {}):
    conditions = [manifold, distType, kfunc, nn, fitVersion, nballs, spacing]
    adapted = {}
    for key in mses.keys():
        match = True
        for j, setting in enumerate(conditions):
            if (setting != -1) and (key[j] != setting):
                match = False
        if match:
            adapted[key] = mses[key]
    return adapted

# AAAH figure out function for getting multiple param combos
# sort tuples by error to get combos sorted from lowest to highest in output

def two_params_sorted(p1, p2, mse):
    errors = []
    vals = unique_conditions(mse)
    stuff = [-1, -1, -1, -1, -1, -1, -1]
    conditions = ['manifold', 'distType', 'kfunc', 'nn', 'fitVersion', 'nballs', 'spacing']
    param1 = conditions[p1]
    param2 = conditions[p2]
    print("average error by", param1, "and", param2)
    for i in vals[param1]:
        for j in vals[param2]:
            stuff[p1] = i
            stuff[p2] = j
            manifold, distType, kfunc, nn, fitVersion, nballs, spacing = stuff
            error = avgmse_condition(manifold, distType, kfunc, nn, fitVersion, nballs, spacing, mse)
            errors.append((i, j, error))
            # print(i, j, error)
    errors.sort(key=lambda y: y[2])
    for item in errors:
        print(item)
    return None

'''
new1 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse1)
new2 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse2)
new3 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse3)
new4 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse4)
new5 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse5)
new6 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse6)
new7 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse7)
new8 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse8)
new9 = adapt_mse(0, 0, 3, 20, 1, -1, -1, mse9)

allmses = [new1, new2, new3, new4, new5, new6, new7, new8, new9]

i = 1
for item in allmses:
    print('NEW MSE', i)
    print(unique_conditions(item))
    two_params_sorted(5, 6, item)
    i += 1
'''

In [ ]:
mse10 = populate_specific(2000, [0], [0], [3], [20], [1], [25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75], 1)


Issue: So it turns out I ran mse10 in 1 dimension oops
First: check curvature calculations for R dependence

In [ ]:
Xr1 = m.Sphere().sample(10, 2, R = 1)
print(m.Sphere().exact_curvatures(Xr1))

Xr2 = m.Sphere().sample(10, 2, R = 2)
print(m.Sphere().exact_curvatures(Xr2))

Xr3 = m.Sphere().sample(10, 2, R = 3)
print(m.Sphere().exact_curvatures(Xr3))

In [ ]:
mse11 = populate_specific(2000, [0], [0], [3], [20], [1], [25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75], 2)
mse12 = populate_specific(2000, [0], [0], [3], [20], [1], [25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75], 3)

In [ ]:
print(mse10)
print(mse11)
print(mse12)


Results from MSE 10-12 (collapsed)

MSE 10

{(0, 0, 3, 20, 1, 25, 0.01): 714.2318526160279, (0, 0, 3, 20, 1, 25, 0.02): 240.78482293747112, (0, 0, 3, 20, 1, 25, 0.03): 81.12513740131567, (0, 0, 3, 20, 1, 25, 0.04): 30.89365572807702, (0, 0, 3, 20, 1, 25, 0.05): 16.070734776106487, (0, 0, 3, 20, 1, 25, 0.06): 4.96294707607304, (0, 0, 3, 20, 1, 25, 0.07): 2.9534755726691837, (0, 0, 3, 20, 1, 25, 0.08): 1.5994564910407962, (0, 0, 3, 20, 1, 25, 0.09): 0.9743461567488798, (0, 0, 3, 20, 1, 25, 0.1): 0.6254751882723855, (0, 0, 3, 20, 1, 25, 0.11): 0.3113291940117501, (0, 0, 3, 20, 1, 25, 0.12): 0.26042835808254367, (0, 0, 3, 20, 1, 25, 0.13): 0.2226791940558842, (0, 0, 3, 20, 1, 25, 0.14): 0.23011770815865107, (0, 0, 3, 20, 1, 25, 0.15): 0.2925194951712167, (0, 0, 3, 20, 1, 25, 0.16): 0.3629938799649773, (0, 0, 3, 20, 1, 25, 0.17): 0.45941370010110916, (0, 0, 3, 20, 1, 25, 0.18): 0.5571738468324094, (0, 0, 3, 20, 1, 25, 0.19): 0.6646735304168008, (0, 0, 3, 20, 1, 25, 0.2): 0.7908724081520944, (0, 0, 3, 20, 1, 30, 0.01): 333.6767096068961, (0, 0, 3, 20, 1, 30, 0.02): 134.45904815201374, (0, 0, 3, 20, 1, 30, 0.03): 43.79028348615275, (0, 0, 3, 20, 1, 30, 0.04): 13.186668350662803, (0, 0, 3, 20, 1, 30, 0.05): 6.655853655569017, (0, 0, 3, 20, 1, 30, 0.06): 2.5113472061444533, (0, 0, 3, 20, 1, 30, 0.07): 1.0550006272802688, (0, 0, 3, 20, 1, 30, 0.08): 0.5429646267064829, (0, 0, 3, 20, 1, 30, 0.09): 0.3083575293852623, (0, 0, 3, 20, 1, 30, 0.1): 0.28087993241625636, (0, 0, 3, 20, 1, 30, 0.11): 0.22681406598861878, (0, 0, 3, 20, 1, 30, 0.12): 0.24642919697080048, (0, 0, 3, 20, 1, 30, 0.13): 0.34030455907615326, (0, 0, 3, 20, 1, 30, 0.14): 0.44277366105019983, (0, 0, 3, 20, 1, 30, 0.15): 0.559960774155103, (0, 0, 3, 20, 1, 30, 0.16): 0.7057106274784908, (0, 0, 3, 20, 1, 30, 0.17): 0.844247210682293, (0, 0, 3, 20, 1, 30, 0.18): 0.9961483287540466, (0, 0, 3, 20, 1, 30, 0.19): 1.1473036081188777, (0, 0, 3, 20, 1, 30, 0.2): 1.2933514971164823, (0, 0, 3, 20, 1, 35, 0.01): 222.8466568905075, (0, 0, 3, 20, 1, 35, 0.02): 87.13996717367493, (0, 0, 3, 20, 1, 35, 0.03): 23.38976699204365, (0, 0, 3, 20, 1, 35, 0.04): 6.676334163998717, (0, 0, 3, 20, 1, 35, 0.05): 1.6116251305578375, (0, 0, 3, 20, 1, 35, 0.06): 0.9027525542249444, (0, 0, 3, 20, 1, 35, 0.07): 0.471082486672903, (0, 0, 3, 20, 1, 35, 0.08): 0.21717343120981802, (0, 0, 3, 20, 1, 35, 0.09): 0.18358451344832497, (0, 0, 3, 20, 1, 35, 0.1): 0.2550984815260267, (0, 0, 3, 20, 1, 35, 0.11): 0.3241798074824048, (0, 0, 3, 20, 1, 35, 0.12): 0.4504543002866954, (0, 0, 3, 20, 1, 35, 0.13): 0.6005369685143174, (0, 0, 3, 20, 1, 35, 0.14): 0.7585933786514247, (0, 0, 3, 20, 1, 35, 0.15): 0.9270250609979723, (0, 0, 3, 20, 1, 35, 0.16): 1.107311044223501, (0, 0, 3, 20, 1, 35, 0.17): 1.2785585339522318, (0, 0, 3, 20, 1, 35, 0.18): 1.4428858157069886, (0, 0, 3, 20, 1, 35, 0.19): 1.6062744281236578, (0, 0, 3, 20, 1, 35, 0.2): 1.750366867230346, (0, 0, 3, 20, 1, 40, 0.01): 206.09774039641115, (0, 0, 3, 20, 1, 40, 0.02): 43.429284426925484, (0, 0, 3, 20, 1, 40, 0.03): 11.250213445644444, (0, 0, 3, 20, 1, 40, 0.04): 3.090546587300602, (0, 0, 3, 20, 1, 40, 0.05): 1.3989721650703781, (0, 0, 3, 20, 1, 40, 0.06): 0.3027187075004106, (0, 0, 3, 20, 1, 40, 0.07): 0.23264153537851423, (0, 0, 3, 20, 1, 40, 0.08): 0.2189781376156674, (0, 0, 3, 20, 1, 40, 0.09): 0.2587107273933974, (0, 0, 3, 20, 1, 40, 0.1): 0.3778564797389976, (0, 0, 3, 20, 1, 40, 0.11): 0.5213993560653817, (0, 0, 3, 20, 1, 40, 0.12): 0.7163229623760153, (0, 0, 3, 20, 1, 40, 0.13): 0.9112783615242358, (0, 0, 3, 20, 1, 40, 0.14): 1.1147923257889114, (0, 0, 3, 20, 1, 40, 0.15): 1.3080714150069999, (0, 0, 3, 20, 1, 40, 0.16): 1.4967795830002002, (0, 0, 3, 20, 1, 40, 0.17): 1.6708854636075836, (0, 0, 3, 20, 1, 40, 0.18): 1.8367213151112685, (0, 0, 3, 20, 1, 40, 0.19): 1.9874262360387842, (0, 0, 3, 20, 1, 40, 0.2): 2.126119138178059, (0, 0, 3, 20, 1, 45, 0.01): 138.71471399876188, (0, 0, 3, 20, 1, 45, 0.02): 24.517245222625032, (0, 0, 3, 20, 1, 45, 0.03): 6.7679479530473525, (0, 0, 3, 20, 1, 45, 0.04): 2.01965545066279, (0, 0, 3, 20, 1, 45, 0.05): 0.6661416812887843, (0, 0, 3, 20, 1, 45, 0.06): 0.23197523100776965, (0, 0, 3, 20, 1, 45, 0.07): 0.1979441823213399, (0, 0, 3, 20, 1, 45, 0.08): 0.25417002399419186, (0, 0, 3, 20, 1, 45, 0.09): 0.3994999961645833, (0, 0, 3, 20, 1, 45, 0.1): 0.5819332919508357, (0, 0, 3, 20, 1, 45, 0.11): 0.7905763244651483, (0, 0, 3, 20, 1, 45, 0.12): 1.0224413566501884, (0, 0, 3, 20, 1, 45, 0.13): 1.2463228226189267, (0, 0, 3, 20, 1, 45, 0.14): 1.4558985568505851, (0, 0, 3, 20, 1, 45, 0.15): 1.6529259758264465, (0, 0, 3, 20, 1, 45, 0.16): 1.8406830163662897, (0, 0, 3, 20, 1, 45, 0.17): 2.011006187606818, (0, 0, 3, 20, 1, 45, 0.18): 2.16578798421945, (0, 0, 3, 20, 1, 45, 0.19): 2.3039574048797977, (0, 0, 3, 20, 1, 45, 0.2): 2.4295225605001844, (0, 0, 3, 20, 1, 50, 0.01): 101.34497134209393, (0, 0, 3, 20, 1, 50, 0.02): 19.501698418823285, (0, 0, 3, 20, 1, 50, 0.03): 3.3112794853038134, (0, 0, 3, 20, 1, 50, 0.04): 0.7038331561145184, (0, 0, 3, 20, 1, 50, 0.05): 0.30756353362895295, (0, 0, 3, 20, 1, 50, 0.06): 0.21214732446721105, (0, 0, 3, 20, 1, 50, 0.07): 0.24520656194264848, (0, 0, 3, 20, 1, 50, 0.08): 0.3675669485803636, (0, 0, 3, 20, 1, 50, 0.09): 0.588565551395656, (0, 0, 3, 20, 1, 50, 0.1): 0.8134783188993631, (0, 0, 3, 20, 1, 50, 0.11): 1.0744906815018593, (0, 0, 3, 20, 1, 50, 0.12): 1.319539380606025, (0, 0, 3, 20, 1, 50, 0.13): 1.553321058541564, (0, 0, 3, 20, 1, 50, 0.14): 1.7661072803784514, (0, 0, 3, 20, 1, 50, 0.15): 1.9572121809583145, (0, 0, 3, 20, 1, 50, 0.16): 2.134116659977966, (0, 0, 3, 20, 1, 50, 0.17): 2.2930271715988217, (0, 0, 3, 20, 1, 50, 0.18): 2.432940772068939, (0, 0, 3, 20, 1, 50, 0.19): 2.5603466115942064, (0, 0, 3, 20, 1, 50, 0.2): 2.6736395801802932, (0, 0, 3, 20, 1, 55, 0.01): 86.12347330200032, (0, 0, 3, 20, 1, 55, 0.02): 11.21733685747141, (0, 0, 3, 20, 1, 55, 0.03): 2.3783862227134294, (0, 0, 3, 20, 1, 55, 0.04): 0.5688049422203969, (0, 0, 3, 20, 1, 55, 0.05): 0.16230285108614045, (0, 0, 3, 20, 1, 55, 0.06): 0.20366842762519113, (0, 0, 3, 20, 1, 55, 0.07): 0.33456104372161416, (0, 0, 3, 20, 1, 55, 0.08): 0.548300882508124, (0, 0, 3, 20, 1, 55, 0.09): 0.804629734806942, (0, 0, 3, 20, 1, 55, 0.1): 1.0764761713219262, (0, 0, 3, 20, 1, 55, 0.11): 1.3538829559756178, (0, 0, 3, 20, 1, 55, 0.12): 1.5976630431850822, (0, 0, 3, 20, 1, 55, 0.13): 1.8286858706520617, (0, 0, 3, 20, 1, 55, 0.14): 2.0382086918651208, (0, 0, 3, 20, 1, 55, 0.15): 2.2190157095168894, (0, 0, 3, 20, 1, 55, 0.16): 2.3808267617102192, (0, 0, 3, 20, 1, 55, 0.17): 2.5260029844392093, (0, 0, 3, 20, 1, 55, 0.18): 2.653036748178499, (0, 0, 3, 20, 1, 55, 0.19): 2.767012937467251, (0, 0, 3, 20, 1, 55, 0.2): 2.8678099901044867, (0, 0, 3, 20, 1, 60, 0.01): 60.680602690307694, (0, 0, 3, 20, 1, 60, 0.02): 6.043298649338753, (0, 0, 3, 20, 1, 60, 0.03): 1.1371781734197794, (0, 0, 3, 20, 1, 60, 0.04): 0.3389088481473383, (0, 0, 3, 20, 1, 60, 0.05): 0.18404598888352552, (0, 0, 3, 20, 1, 60, 0.06): 0.24849646966777345, (0, 0, 3, 20, 1, 60, 0.07): 0.4547921795539099, (0, 0, 3, 20, 1, 60, 0.08): 0.7339343363991047, (0, 0, 3, 20, 1, 60, 0.09): 1.0342554272405684, (0, 0, 3, 20, 1, 60, 0.1): 1.326279964620258, (0, 0, 3, 20, 1, 60, 0.11): 1.6039143851238782, (0, 0, 3, 20, 1, 60, 0.12): 1.84995778848202, (0, 0, 3, 20, 1, 60, 0.13): 2.0747577563791015, (0, 0, 3, 20, 1, 60, 0.14): 2.2662199219738817, (0, 0, 3, 20, 1, 60, 0.15): 2.4381149527439328, (0, 0, 3, 20, 1, 60, 0.16): 2.589285138901148, (0, 0, 3, 20, 1, 60, 0.17): 2.719121191214134, (0, 0, 3, 20, 1, 60, 0.18): 2.833501523948611, (0, 0, 3, 20, 1, 60, 0.19): 2.9354030788280374, (0, 0, 3, 20, 1, 60, 0.2): 3.025296053150464, (0, 0, 3, 20, 1, 65, 0.01): 47.35836543725949, (0, 0, 3, 20, 1, 65, 0.02): 5.104742695637182, (0, 0, 3, 20, 1, 65, 0.03): 0.6581374242055137, (0, 0, 3, 20, 1, 65, 0.04): 0.22941070246902381, (0, 0, 3, 20, 1, 65, 0.05): 0.20614644370379973, (0, 0, 3, 20, 1, 65, 0.06): 0.3532061316804556, (0, 0, 3, 20, 1, 65, 0.07): 0.6222944275600538, (0, 0, 3, 20, 1, 65, 0.08): 0.9355152188854704, (0, 0, 3, 20, 1, 65, 0.09): 1.256033329608622, (0, 0, 3, 20, 1, 65, 0.1): 1.5636537054097153, (0, 0, 3, 20, 1, 65, 0.11): 1.8340836025304046, (0, 0, 3, 20, 1, 65, 0.12): 2.0747339416064405, (0, 0, 3, 20, 1, 65, 0.13): 2.286052434776556, (0, 0, 3, 20, 1, 65, 0.14): 2.4682790828253816, (0, 0, 3, 20, 1, 65, 0.15): 2.6249393227034985, (0, 0, 3, 20, 1, 65, 0.16): 2.760526021707889, (0, 0, 3, 20, 1, 65, 0.17): 2.878687986436891, (0, 0, 3, 20, 1, 65, 0.18): 2.98297772913061, (0, 0, 3, 20, 1, 65, 0.19): 3.073137581189994, (0, 0, 3, 20, 1, 65, 0.2): 3.1527642309437143, (0, 0, 3, 20, 1, 70, 0.01): 34.45269811892231, (0, 0, 3, 20, 1, 70, 0.02): 3.005173011951808, (0, 0, 3, 20, 1, 70, 0.03): 0.7137720274079358, (0, 0, 3, 20, 1, 70, 0.04): 0.21345673479200758, (0, 0, 3, 20, 1, 70, 0.05): 0.23440955460743482, (0, 0, 3, 20, 1, 70, 0.06): 0.471877301180506, (0, 0, 3, 20, 1, 70, 0.07): 0.778746035388576, (0, 0, 3, 20, 1, 70, 0.08): 1.1401718149972266, (0, 0, 3, 20, 1, 70, 0.09): 1.4722195248348107, (0, 0, 3, 20, 1, 70, 0.1): 1.7736674343253958, (0, 0, 3, 20, 1, 70, 0.11): 2.0436387254591284, (0, 0, 3, 20, 1, 70, 0.12): 2.271286969226115, (0, 0, 3, 20, 1, 70, 0.13): 2.4680393009881083, (0, 0, 3, 20, 1, 70, 0.14): 2.635835822554517, (0, 0, 3, 20, 1, 70, 0.15): 2.7808323417161347, (0, 0, 3, 20, 1, 70, 0.16): 2.905594498662871, (0, 0, 3, 20, 1, 70, 0.17): 3.013117886722699, (0, 0, 3, 20, 1, 70, 0.18): 3.1065850404486848, (0, 0, 3, 20, 1, 70, 0.19): 3.1876546706140956, (0, 0, 3, 20, 1, 70, 0.2): 3.258095509774066, (0, 0, 3, 20, 1, 75, 0.01): 25.947335692437047, (0, 0, 3, 20, 1, 75, 0.02): 2.1543133550793634, (0, 0, 3, 20, 1, 75, 0.03): 0.30272025356606425, (0, 0, 3, 20, 1, 75, 0.04): 0.1495072841271193, (0, 0, 3, 20, 1, 75, 0.05): 0.3086322891452753, (0, 0, 3, 20, 1, 75, 0.06): 0.5905320700882943, (0, 0, 3, 20, 1, 75, 0.07): 0.9663856813742663, (0, 0, 3, 20, 1, 75, 0.08): 1.3306682293742687, (0, 0, 3, 20, 1, 75, 0.09): 1.6785810218058546, (0, 0, 3, 20, 1, 75, 0.1): 1.9708373808538382, (0, 0, 3, 20, 1, 75, 0.11): 2.2252131745297454, (0, 0, 3, 20, 1, 75, 0.12): 2.443029266997387, (0, 0, 3, 20, 1, 75, 0.13): 2.6272015502115766, (0, 0, 3, 20, 1, 75, 0.14): 2.782640590970321, (0, 0, 3, 20, 1, 75, 0.15): 2.9151722631997536, (0, 0, 3, 20, 1, 75, 0.16): 3.0285626509499886, (0, 0, 3, 20, 1, 75, 0.17): 3.1247898872734288, (0, 0, 3, 20, 1, 75, 0.18): 3.209543006952975, (0, 0, 3, 20, 1, 75, 0.19): 3.2823635441205297, (0, 0, 3, 20, 1, 75, 0.2): 3.3456726061826174}

MSE 11

{(0, 0, 3, 20, 1, 25, 0.01): 2726.4282624748603, (0, 0, 3, 20, 1, 25, 0.02): 318.44658743686165, (0, 0, 3, 20, 1, 25, 0.03): 105.58429229942247, (0, 0, 3, 20, 1, 25, 0.04): 38.62552188501312, (0, 0, 3, 20, 1, 25, 0.05): 24.44101109452838, (0, 0, 3, 20, 1, 25, 0.06): 10.353003942845554, (0, 0, 3, 20, 1, 25, 0.07): 6.222234104634132, (0, 0, 3, 20, 1, 25, 0.08): 3.9128027695076315, (0, 0, 3, 20, 1, 25, 0.09): 2.2279161421550353, (0, 0, 3, 20, 1, 25, 0.1): 1.4527796941271618, (0, 0, 3, 20, 1, 25, 0.11): 1.0693440753924426, (0, 0, 3, 20, 1, 25, 0.12): 0.6549883674879613, (0, 0, 3, 20, 1, 25, 0.13): 0.419486072983222, (0, 0, 3, 20, 1, 25, 0.14): 0.2999775963315656, (0, 0, 3, 20, 1, 25, 0.15): 0.1949704266361804, (0, 0, 3, 20, 1, 25, 0.16): 0.1837618010761071, (0, 0, 3, 20, 1, 25, 0.17): 0.130453052415364, (0, 0, 3, 20, 1, 25, 0.18): 0.07433827416892069, (0, 0, 3, 20, 1, 25, 0.19): 0.06300670959108877, (0, 0, 3, 20, 1, 25, 0.2): 0.0386295972589639, (0, 0, 3, 20, 1, 30, 0.01): 1217.2975048645733, (0, 0, 3, 20, 1, 30, 0.02): 175.3679146576115, (0, 0, 3, 20, 1, 30, 0.03): 51.48075836614856, (0, 0, 3, 20, 1, 30, 0.04): 19.07681199929053, (0, 0, 3, 20, 1, 30, 0.05): 10.8296525077907, (0, 0, 3, 20, 1, 30, 0.06): 4.883247057761804, (0, 0, 3, 20, 1, 30, 0.07): 2.881837866574835, (0, 0, 3, 20, 1, 30, 0.08): 1.45077355903142, (0, 0, 3, 20, 1, 30, 0.09): 0.9081487342284251, (0, 0, 3, 20, 1, 30, 0.1): 0.608286317724251, (0, 0, 3, 20, 1, 30, 0.11): 0.3624489738318422, (0, 0, 3, 20, 1, 30, 0.12): 0.22459304259163018, (0, 0, 3, 20, 1, 30, 0.13): 0.13667881193120338, (0, 0, 3, 20, 1, 30, 0.14): 0.10666802842518139, (0, 0, 3, 20, 1, 30, 0.15): 0.06810633472733088, (0, 0, 3, 20, 1, 30, 0.16): 0.05002793141360611, (0, 0, 3, 20, 1, 30, 0.17): 0.03436160767401392, (0, 0, 3, 20, 1, 30, 0.18): 0.022503877036364875, (0, 0, 3, 20, 1, 30, 0.19): 0.015509134686352026, (0, 0, 3, 20, 1, 30, 0.2): 0.010591147456547599, (0, 0, 3, 20, 1, 35, 0.01): 791.4892301912724, (0, 0, 3, 20, 1, 35, 0.02): 105.8808501954962, (0, 0, 3, 20, 1, 35, 0.03): 29.797833234215798, (0, 0, 3, 20, 1, 35, 0.04): 12.7931237525057, (0, 0, 3, 20, 1, 35, 0.05): 5.434655266805519, (0, 0, 3, 20, 1, 35, 0.06): 2.060357173492649, (0, 0, 3, 20, 1, 35, 0.07): 1.312249294621182, (0, 0, 3, 20, 1, 35, 0.08): 0.6347373957662984, (0, 0, 3, 20, 1, 35, 0.09): 0.415087089795585, (0, 0, 3, 20, 1, 35, 0.1): 0.21951637062140178, (0, 0, 3, 20, 1, 35, 0.11): 0.1473567970767424, (0, 0, 3, 20, 1, 35, 0.12): 0.10114368538099328, (0, 0, 3, 20, 1, 35, 0.13): 0.05776550233249112, (0, 0, 3, 20, 1, 35, 0.14): 0.037934393184401956, (0, 0, 3, 20, 1, 35, 0.15): 0.022453370705009378, (0, 0, 3, 20, 1, 35, 0.16): 0.013931087610408273, (0, 0, 3, 20, 1, 35, 0.17): 0.010009790807710492, (0, 0, 3, 20, 1, 35, 0.18): 0.009185883253770743, (0, 0, 3, 20, 1, 35, 0.19): 0.007522765522868905, (0, 0, 3, 20, 1, 35, 0.2): 0.008126263626242851, (0, 0, 3, 20, 1, 40, 0.01): 484.91214082533014, (0, 0, 3, 20, 1, 40, 0.02): 78.10912283832853, (0, 0, 3, 20, 1, 40, 0.03): 16.03340141640151, (0, 0, 3, 20, 1, 40, 0.04): 5.894261744337176, (0, 0, 3, 20, 1, 40, 0.05): 2.423752349413845, (0, 0, 3, 20, 1, 40, 0.06): 1.2300415990042588, (0, 0, 3, 20, 1, 40, 0.07): 0.5653571189338067, (0, 0, 3, 20, 1, 40, 0.08): 0.28910425514861626, (0, 0, 3, 20, 1, 40, 0.09): 0.18598885664080428, (0, 0, 3, 20, 1, 40, 0.1): 0.11037371418610388, (0, 0, 3, 20, 1, 40, 0.11): 0.06698338611852193, (0, 0, 3, 20, 1, 40, 0.12): 0.04847842312597729, (0, 0, 3, 20, 1, 40, 0.13): 0.02454939730324674, (0, 0, 3, 20, 1, 40, 0.14): 0.012265024207244834, (0, 0, 3, 20, 1, 40, 0.15): 0.00824412691922426, (0, 0, 3, 20, 1, 40, 0.16): 0.006935998657993217, (0, 0, 3, 20, 1, 40, 0.17): 0.008107878511676363, (0, 0, 3, 20, 1, 40, 0.18): 0.009887950991347604, (0, 0, 3, 20, 1, 40, 0.19): 0.011559630248267509, (0, 0, 3, 20, 1, 40, 0.2): 0.01646503070603253, (0, 0, 3, 20, 1, 45, 0.01): 331.32703769949615, (0, 0, 3, 20, 1, 45, 0.02): 53.680352251783276, (0, 0, 3, 20, 1, 45, 0.03): 10.855344191780159, (0, 0, 3, 20, 1, 45, 0.04): 3.1335843845848075, (0, 0, 3, 20, 1, 45, 0.05): 1.3563140292357276, (0, 0, 3, 20, 1, 45, 0.06): 0.6387554697558705, (0, 0, 3, 20, 1, 45, 0.07): 0.36329789703310245, (0, 0, 3, 20, 1, 45, 0.08): 0.1849853776865425, (0, 0, 3, 20, 1, 45, 0.09): 0.0976489280903036, (0, 0, 3, 20, 1, 45, 0.1): 0.04389999557057662, (0, 0, 3, 20, 1, 45, 0.11): 0.023605497501381193, (0, 0, 3, 20, 1, 45, 0.12): 0.018236696998389354, (0, 0, 3, 20, 1, 45, 0.13): 0.007862041147030643, (0, 0, 3, 20, 1, 45, 0.14): 0.00694940885043103, (0, 0, 3, 20, 1, 45, 0.15): 0.007571543628239237, (0, 0, 3, 20, 1, 45, 0.16): 0.00949943330974212, (0, 0, 3, 20, 1, 45, 0.17): 0.01432024576838009, (0, 0, 3, 20, 1, 45, 0.18): 0.018150515904091363, (0, 0, 3, 20, 1, 45, 0.19): 0.023606961571811146, (0, 0, 3, 20, 1, 45, 0.2): 0.030906191680055154, (0, 0, 3, 20, 1, 50, 0.01): 248.8192415974912, (0, 0, 3, 20, 1, 50, 0.02): 30.014158884516785, (0, 0, 3, 20, 1, 50, 0.03): 6.504671586293825, (0, 0, 3, 20, 1, 50, 0.04): 2.4460961740685803, (0, 0, 3, 20, 1, 50, 0.05): 0.8951982043876607, (0, 0, 3, 20, 1, 50, 0.06): 0.4637097144875303, (0, 0, 3, 20, 1, 50, 0.07): 0.16918590277204004, (0, 0, 3, 20, 1, 50, 0.08): 0.08066455119069925, (0, 0, 3, 20, 1, 50, 0.09): 0.05204390829964261, (0, 0, 3, 20, 1, 50, 0.1): 0.02804682732181513, (0, 0, 3, 20, 1, 50, 0.11): 0.012145471942746192, (0, 0, 3, 20, 1, 50, 0.12): 0.008395829749769577, (0, 0, 3, 20, 1, 50, 0.13): 0.006344012109814914, (0, 0, 3, 20, 1, 50, 0.14): 0.006746423331903903, (0, 0, 3, 20, 1, 50, 0.15): 0.012624946479268068, (0, 0, 3, 20, 1, 50, 0.16): 0.01743005426614811, (0, 0, 3, 20, 1, 50, 0.17): 0.02297098028340159, (0, 0, 3, 20, 1, 50, 0.18): 0.03059784718868274, (0, 0, 3, 20, 1, 50, 0.19): 0.03937549114584424, (0, 0, 3, 20, 1, 50, 0.2): 0.04604396076963275, (0, 0, 3, 20, 1, 55, 0.01): 167.26846235650842, (0, 0, 3, 20, 1, 55, 0.02): 21.348526899732978, (0, 0, 3, 20, 1, 55, 0.03): 3.8566333657792957, (0, 0, 3, 20, 1, 55, 0.04): 1.3660617682727692, (0, 0, 3, 20, 1, 55, 0.05): 0.6721787216283788, (0, 0, 3, 20, 1, 55, 0.06): 0.2638888780532395, (0, 0, 3, 20, 1, 55, 0.07): 0.10539634742067304, (0, 0, 3, 20, 1, 55, 0.08): 0.05068569059405405, (0, 0, 3, 20, 1, 55, 0.09): 0.02572000077551368, (0, 0, 3, 20, 1, 55, 0.1): 0.011991072383058885, (0, 0, 3, 20, 1, 55, 0.11): 0.00636260578005052, (0, 0, 3, 20, 1, 55, 0.12): 0.007522954771822718, (0, 0, 3, 20, 1, 55, 0.13): 0.009487836340027011, (0, 0, 3, 20, 1, 55, 0.14): 0.013596344242855737, (0, 0, 3, 20, 1, 55, 0.15): 0.020599872140356787, (0, 0, 3, 20, 1, 55, 0.16): 0.02846230688702624, (0, 0, 3, 20, 1, 55, 0.17): 0.03551021571863835, (0, 0, 3, 20, 1, 55, 0.18): 0.045179900446573044, (0, 0, 3, 20, 1, 55, 0.19): 0.05460547853284441, (0, 0, 3, 20, 1, 55, 0.2): 0.0635014078155783, (0, 0, 3, 20, 1, 60, 0.01): 119.86883634352698, (0, 0, 3, 20, 1, 60, 0.02): 14.731360346489302, (0, 0, 3, 20, 1, 60, 0.03): 3.1980025325868833, (0, 0, 3, 20, 1, 60, 0.04): 0.9772284696054728, (0, 0, 3, 20, 1, 60, 0.05): 0.3420851089358831, (0, 0, 3, 20, 1, 60, 0.06): 0.12992072368220461, (0, 0, 3, 20, 1, 60, 0.07): 0.07786359981705311, (0, 0, 3, 20, 1, 60, 0.08): 0.03296987046878534, (0, 0, 3, 20, 1, 60, 0.09): 0.011907648931403269, (0, 0, 3, 20, 1, 60, 0.1): 0.005988867061248519, (0, 0, 3, 20, 1, 60, 0.11): 0.007435483306114451, (0, 0, 3, 20, 1, 60, 0.12): 0.008350447513890021, (0, 0, 3, 20, 1, 60, 0.13): 0.01630453651166262, (0, 0, 3, 20, 1, 60, 0.14): 0.024132648986033133, (0, 0, 3, 20, 1, 60, 0.15): 0.03097103780898766, (0, 0, 3, 20, 1, 60, 0.16): 0.04114478303102903, (0, 0, 3, 20, 1, 60, 0.17): 0.05083917077952614, (0, 0, 3, 20, 1, 60, 0.18): 0.05968978774632178, (0, 0, 3, 20, 1, 60, 0.19): 0.06981290559480421, (0, 0, 3, 20, 1, 60, 0.2): 0.07970833641209898, (0, 0, 3, 20, 1, 65, 0.01): 98.66878824499192, (0, 0, 3, 20, 1, 65, 0.02): 12.074842996831958, (0, 0, 3, 20, 1, 65, 0.03): 2.4376617002136345, (0, 0, 3, 20, 1, 65, 0.04): 0.6810691298290099, (0, 0, 3, 20, 1, 65, 0.05): 0.24326908582471674, (0, 0, 3, 20, 1, 65, 0.06): 0.09426746089915211, (0, 0, 3, 20, 1, 65, 0.07): 0.03422115255226056, (0, 0, 3, 20, 1, 65, 0.08): 0.011553418415921718, (0, 0, 3, 20, 1, 65, 0.09): 0.006367673645517341, (0, 0, 3, 20, 1, 65, 0.1): 0.003888774346853614, (0, 0, 3, 20, 1, 65, 0.11): 0.008592435268471844, (0, 0, 3, 20, 1, 65, 0.12): 0.015140986622807046, (0, 0, 3, 20, 1, 65, 0.13): 0.023416462142362947, (0, 0, 3, 20, 1, 65, 0.14): 0.03297096035623661, (0, 0, 3, 20, 1, 65, 0.15): 0.04349305205896686, (0, 0, 3, 20, 1, 65, 0.16): 0.05425328514487462, (0, 0, 3, 20, 1, 65, 0.17): 0.06442833359707661, (0, 0, 3, 20, 1, 65, 0.18): 0.0754144862751853, (0, 0, 3, 20, 1, 65, 0.19): 0.08530512747681569, (0, 0, 3, 20, 1, 65, 0.2): 0.09537975019843294, (0, 0, 3, 20, 1, 70, 0.01): 76.91428565496088, (0, 0, 3, 20, 1, 70, 0.02): 6.408488004704876, (0, 0, 3, 20, 1, 70, 0.03): 1.4912809861327505, (0, 0, 3, 20, 1, 70, 0.04): 0.31978590243409355, (0, 0, 3, 20, 1, 70, 0.05): 0.17600859755586287, (0, 0, 3, 20, 1, 70, 0.06): 0.04975973401068208, (0, 0, 3, 20, 1, 70, 0.07): 0.018311178415093023, (0, 0, 3, 20, 1, 70, 0.08): 0.010007875676026628, (0, 0, 3, 20, 1, 70, 0.09): 0.0051314553921075435, (0, 0, 3, 20, 1, 70, 0.1): 0.007817244432534376, (0, 0, 3, 20, 1, 70, 0.11): 0.013755485581782806, (0, 0, 3, 20, 1, 70, 0.12): 0.022343889564237364, (0, 0, 3, 20, 1, 70, 0.13): 0.03332409472832784, (0, 0, 3, 20, 1, 70, 0.14): 0.04382365772375225, (0, 0, 3, 20, 1, 70, 0.15): 0.05589458369191053, (0, 0, 3, 20, 1, 70, 0.16): 0.06732589397690722, (0, 0, 3, 20, 1, 70, 0.17): 0.07897070062450307, (0, 0, 3, 20, 1, 70, 0.18): 0.08906142541699634, (0, 0, 3, 20, 1, 70, 0.19): 0.09903773426404362, (0, 0, 3, 20, 1, 70, 0.2): 0.10891694773318901, (0, 0, 3, 20, 1, 75, 0.01): 69.83614349557966, (0, 0, 3, 20, 1, 75, 0.02): 4.864223786527774, (0, 0, 3, 20, 1, 75, 0.03): 1.2270995628090053, (0, 0, 3, 20, 1, 75, 0.04): 0.3745311609698748, (0, 0, 3, 20, 1, 75, 0.05): 0.1212058551755562, (0, 0, 3, 20, 1, 75, 0.06): 0.03130009072442116, (0, 0, 3, 20, 1, 75, 0.07): 0.01038971986514709, (0, 0, 3, 20, 1, 75, 0.08): 0.0038662466058983704, (0, 0, 3, 20, 1, 75, 0.09): 0.007207598229961277, (0, 0, 3, 20, 1, 75, 0.1): 0.01092724289092143, (0, 0, 3, 20, 1, 75, 0.11): 0.020034374176233966, (0, 0, 3, 20, 1, 75, 0.12): 0.03199235943516538, (0, 0, 3, 20, 1, 75, 0.13): 0.04393647526799235, (0, 0, 3, 20, 1, 75, 0.14): 0.05532305236619686, (0, 0, 3, 20, 1, 75, 0.15): 0.06787880560537958, (0, 0, 3, 20, 1, 75, 0.16): 0.08051950028157144, (0, 0, 3, 20, 1, 75, 0.17): 0.09173860062118425, (0, 0, 3, 20, 1, 75, 0.18): 0.10259629331772326, (0, 0, 3, 20, 1, 75, 0.19): 0.11199898427920572, (0, 0, 3, 20, 1, 75, 0.2): 0.12155306506782883}

MSE 12

{(0, 0, 3, 20, 1, 25, 0.01): 5199.059482207876, (0, 0, 3, 20, 1, 25, 0.02): 977.9152968937474, (0, 0, 3, 20, 1, 25, 0.03): 297.46804957361473, (0, 0, 3, 20, 1, 25, 0.04): 110.68230697088694, (0, 0, 3, 20, 1, 25, 0.05): 53.70433584411534, (0, 0, 3, 20, 1, 25, 0.06): 27.775830303199896, (0, 0, 3, 20, 1, 25, 0.07): 16.405098373650773, (0, 0, 3, 20, 1, 25, 0.08): 9.420964174246299, (0, 0, 3, 20, 1, 25, 0.09): 5.718178480986846, (0, 0, 3, 20, 1, 25, 0.1): 3.87864812697794, (0, 0, 3, 20, 1, 25, 0.11): 2.8331542194164356, (0, 0, 3, 20, 1, 25, 0.12): 1.9756780336115425, (0, 0, 3, 20, 1, 25, 0.13): 1.391670210388825, (0, 0, 3, 20, 1, 25, 0.14): 0.9757087621462993, (0, 0, 3, 20, 1, 25, 0.15): 0.7822248591949315, (0, 0, 3, 20, 1, 25, 0.16): 0.6126427005830604, (0, 0, 3, 20, 1, 25, 0.17): 0.42378099269874075, (0, 0, 3, 20, 1, 25, 0.18): 0.3283715182016758, (0, 0, 3, 20, 1, 25, 0.19): 0.2662821961595403, (0, 0, 3, 20, 1, 25, 0.2): 0.1999093631413159, (0, 0, 3, 20, 1, 30, 0.01): 3590.306087978854, (0, 0, 3, 20, 1, 30, 0.02): 553.7757230139846, (0, 0, 3, 20, 1, 30, 0.03): 158.49152467382046, (0, 0, 3, 20, 1, 30, 0.04): 60.35252010064774, (0, 0, 3, 20, 1, 30, 0.05): 26.27565285468648, (0, 0, 3, 20, 1, 30, 0.06): 12.666878132535977, (0, 0, 3, 20, 1, 30, 0.07): 7.275332571491046, (0, 0, 3, 20, 1, 30, 0.08): 4.593885313694673, (0, 0, 3, 20, 1, 30, 0.09): 2.916390193825856, (0, 0, 3, 20, 1, 30, 0.1): 1.8580725754149618, (0, 0, 3, 20, 1, 30, 0.11): 1.250560619487186, (0, 0, 3, 20, 1, 30, 0.12): 0.8727601753927715, (0, 0, 3, 20, 1, 30, 0.13): 0.6006318713352439, (0, 0, 3, 20, 1, 30, 0.14): 0.4669166185914371, (0, 0, 3, 20, 1, 30, 0.15): 0.32073973012955065, (0, 0, 3, 20, 1, 30, 0.16): 0.25024437339727174, (0, 0, 3, 20, 1, 30, 0.17): 0.19422725060092302, (0, 0, 3, 20, 1, 30, 0.18): 0.13180422231754335, (0, 0, 3, 20, 1, 30, 0.19): 0.10535667742658082, (0, 0, 3, 20, 1, 30, 0.2): 0.07682599750487443, (0, 0, 3, 20, 1, 35, 0.01): 2588.8636629614516, (0, 0, 3, 20, 1, 35, 0.02): 353.01892686498394, (0, 0, 3, 20, 1, 35, 0.03): 99.3862110678656, (0, 0, 3, 20, 1, 35, 0.04): 33.402812809927035, (0, 0, 3, 20, 1, 35, 0.05): 13.872729657915418, (0, 0, 3, 20, 1, 35, 0.06): 6.695211085301595, (0, 0, 3, 20, 1, 35, 0.07): 3.9939984659764884, (0, 0, 3, 20, 1, 35, 0.08): 2.321583268524592, (0, 0, 3, 20, 1, 35, 0.09): 1.414193064904869, (0, 0, 3, 20, 1, 35, 0.1): 0.8645741631710852, (0, 0, 3, 20, 1, 35, 0.11): 0.5983538315107765, (0, 0, 3, 20, 1, 35, 0.12): 0.4156946635339565, (0, 0, 3, 20, 1, 35, 0.13): 0.26691604386026835, (0, 0, 3, 20, 1, 35, 0.14): 0.2025687415434943, (0, 0, 3, 20, 1, 35, 0.15): 0.14765916865640435, (0, 0, 3, 20, 1, 35, 0.16): 0.10730804691272874, (0, 0, 3, 20, 1, 35, 0.17): 0.08070252528245678, (0, 0, 3, 20, 1, 35, 0.18): 0.06030164892206564, (0, 0, 3, 20, 1, 35, 0.19): 0.040178144188263855, (0, 0, 3, 20, 1, 35, 0.2): 0.028817951344081914, (0, 0, 3, 20, 1, 40, 0.01): 1888.7442378521248, (0, 0, 3, 20, 1, 40, 0.02): 228.4542919441646, (0, 0, 3, 20, 1, 40, 0.03): 59.51009414854098, (0, 0, 3, 20, 1, 40, 0.04): 20.065400777485323, (0, 0, 3, 20, 1, 40, 0.05): 9.577746830938121, (0, 0, 3, 20, 1, 40, 0.06): 4.508059473364868, (0, 0, 3, 20, 1, 40, 0.07): 2.4131197187380047, (0, 0, 3, 20, 1, 40, 0.08): 1.2975789750518465, (0, 0, 3, 20, 1, 40, 0.09): 0.7775215527865621, (0, 0, 3, 20, 1, 40, 0.1): 0.4512084025262858, (0, 0, 3, 20, 1, 40, 0.11): 0.3418572444862232, (0, 0, 3, 20, 1, 40, 0.12): 0.23281135045620102, (0, 0, 3, 20, 1, 40, 0.13): 0.15240978537256936, (0, 0, 3, 20, 1, 40, 0.14): 0.11317680778230761, (0, 0, 3, 20, 1, 40, 0.15): 0.0730660200589307, (0, 0, 3, 20, 1, 40, 0.16): 0.04948469320471921, (0, 0, 3, 20, 1, 40, 0.17): 0.03487062652651804, (0, 0, 3, 20, 1, 40, 0.18): 0.026848983532760412, (0, 0, 3, 20, 1, 40, 0.19): 0.01613583782312467, (0, 0, 3, 20, 1, 40, 0.2): 0.011987148670255594, (0, 0, 3, 20, 1, 45, 0.01): 1256.592226325668, (0, 0, 3, 20, 1, 45, 0.02): 152.76596981585433, (0, 0, 3, 20, 1, 45, 0.03): 35.989742223439336, (0, 0, 3, 20, 1, 45, 0.04): 12.732385995061879, (0, 0, 3, 20, 1, 45, 0.05): 5.451344264421774, (0, 0, 3, 20, 1, 45, 0.06): 2.5777927447155853, (0, 0, 3, 20, 1, 45, 0.07): 1.4187033270361376, (0, 0, 3, 20, 1, 45, 0.08): 0.8307639095167858, (0, 0, 3, 20, 1, 45, 0.09): 0.4679183163952036, (0, 0, 3, 20, 1, 45, 0.1): 0.3016575756175682, (0, 0, 3, 20, 1, 45, 0.11): 0.20191788131798627, (0, 0, 3, 20, 1, 45, 0.12): 0.11803994762620891, (0, 0, 3, 20, 1, 45, 0.13): 0.08203284758756457, (0, 0, 3, 20, 1, 45, 0.14): 0.05091501052418448, (0, 0, 3, 20, 1, 45, 0.15): 0.03669292483375463, (0, 0, 3, 20, 1, 45, 0.16): 0.02377115694365335, (0, 0, 3, 20, 1, 45, 0.17): 0.014116103193327595, (0, 0, 3, 20, 1, 45, 0.18): 0.009940922703131282, (0, 0, 3, 20, 1, 45, 0.19): 0.005700582312038307, (0, 0, 3, 20, 1, 45, 0.2): 0.003275226889041347, (0, 0, 3, 20, 1, 50, 0.01): 940.7880514517127, (0, 0, 3, 20, 1, 50, 0.02): 106.14493035494141, (0, 0, 3, 20, 1, 50, 0.03): 23.56598527946425, (0, 0, 3, 20, 1, 50, 0.04): 7.522335187011906, (0, 0, 3, 20, 1, 50, 0.05): 3.5465867259335817, (0, 0, 3, 20, 1, 50, 0.06): 1.757637463559008, (0, 0, 3, 20, 1, 50, 0.07): 0.9057231880003298, (0, 0, 3, 20, 1, 50, 0.08): 0.46773550080423665, (0, 0, 3, 20, 1, 50, 0.09): 0.297812571283383, (0, 0, 3, 20, 1, 50, 0.1): 0.1843115659402223, (0, 0, 3, 20, 1, 50, 0.11): 0.10726966859703634, (0, 0, 3, 20, 1, 50, 0.12): 0.06916751714283839, (0, 0, 3, 20, 1, 50, 0.13): 0.04649223312507924, (0, 0, 3, 20, 1, 50, 0.14): 0.027341811011730346, (0, 0, 3, 20, 1, 50, 0.15): 0.018781647318446233, (0, 0, 3, 20, 1, 50, 0.16): 0.010761678641137614, (0, 0, 3, 20, 1, 50, 0.17): 0.006238447096594754, (0, 0, 3, 20, 1, 50, 0.18): 0.003927295297286249, (0, 0, 3, 20, 1, 50, 0.19): 0.0016378712497692688, (0, 0, 3, 20, 1, 50, 0.2): 0.0006649935591420564, (0, 0, 3, 20, 1, 55, 0.01): 714.014068385248, (0, 0, 3, 20, 1, 55, 0.02): 68.03767731273634, (0, 0, 3, 20, 1, 55, 0.03): 16.85713702633545, (0, 0, 3, 20, 1, 55, 0.04): 5.9095243682575145, (0, 0, 3, 20, 1, 55, 0.05): 2.5628815239030334, (0, 0, 3, 20, 1, 55, 0.06): 1.1113161782841041, (0, 0, 3, 20, 1, 55, 0.07): 0.5680137774665875, (0, 0, 3, 20, 1, 55, 0.08): 0.31535830986096997, (0, 0, 3, 20, 1, 55, 0.09): 0.18618701853520936, (0, 0, 3, 20, 1, 55, 0.1): 0.10419995493904527, (0, 0, 3, 20, 1, 55, 0.11): 0.0614402209365726, (0, 0, 3, 20, 1, 55, 0.12): 0.038547331334677755, (0, 0, 3, 20, 1, 55, 0.13): 0.02545682726737016, (0, 0, 3, 20, 1, 55, 0.14): 0.015443514393379302, (0, 0, 3, 20, 1, 55, 0.15): 0.008171913337594337, (0, 0, 3, 20, 1, 55, 0.16): 0.004267094818323773, (0, 0, 3, 20, 1, 55, 0.17): 0.0020231749229392645, (0, 0, 3, 20, 1, 55, 0.18): 0.0008472555887934661, (0, 0, 3, 20, 1, 55, 0.19): 0.00026164350602219766, (0, 0, 3, 20, 1, 55, 0.2): 0.00025387499205618814, (0, 0, 3, 20, 1, 60, 0.01): 566.3763656459909, (0, 0, 3, 20, 1, 60, 0.02): 54.368360049277435, (0, 0, 3, 20, 1, 60, 0.03): 12.199590639947651, (0, 0, 3, 20, 1, 60, 0.04): 4.224337856647107, (0, 0, 3, 20, 1, 60, 0.05): 1.6739046381643208, (0, 0, 3, 20, 1, 60, 0.06): 0.7748063679032495, (0, 0, 3, 20, 1, 60, 0.07): 0.4114938971794786, (0, 0, 3, 20, 1, 60, 0.08): 0.20455564217917374, (0, 0, 3, 20, 1, 60, 0.09): 0.12295523535321985, (0, 0, 3, 20, 1, 60, 0.1): 0.07161846037093188, (0, 0, 3, 20, 1, 60, 0.11): 0.0389826270896414, (0, 0, 3, 20, 1, 60, 0.12): 0.02322074311637701, (0, 0, 3, 20, 1, 60, 0.13): 0.011947172213465997, (0, 0, 3, 20, 1, 60, 0.14): 0.006696162665918562, (0, 0, 3, 20, 1, 60, 0.15): 0.0029533764049170593, (0, 0, 3, 20, 1, 60, 0.16): 0.0012886177065743888, (0, 0, 3, 20, 1, 60, 0.17): 0.0004296264530680025, (0, 0, 3, 20, 1, 60, 0.18): 0.00016245005782657176, (0, 0, 3, 20, 1, 60, 0.19): 0.00048647254870863474, (0, 0, 3, 20, 1, 60, 0.2): 0.0012397057362794196, (0, 0, 3, 20, 1, 65, 0.01): 385.83059014364824, (0, 0, 3, 20, 1, 65, 0.02): 39.22968422275738, (0, 0, 3, 20, 1, 65, 0.03): 8.760370175879808, (0, 0, 3, 20, 1, 65, 0.04): 2.5721776664637774, (0, 0, 3, 20, 1, 65, 0.05): 1.1373218447954259, (0, 0, 3, 20, 1, 65, 0.06): 0.5071643406253277, (0, 0, 3, 20, 1, 65, 0.07): 0.2676706292769662, (0, 0, 3, 20, 1, 65, 0.08): 0.14891205267346175, (0, 0, 3, 20, 1, 65, 0.09): 0.07851408735708562, (0, 0, 3, 20, 1, 65, 0.1): 0.04306908225969279, (0, 0, 3, 20, 1, 65, 0.11): 0.0221000039343924, (0, 0, 3, 20, 1, 65, 0.12): 0.012651275265102771, (0, 0, 3, 20, 1, 65, 0.13): 0.005382128008444028, (0, 0, 3, 20, 1, 65, 0.14): 0.0028267055276824044, (0, 0, 3, 20, 1, 65, 0.15): 0.0007945569184816576, (0, 0, 3, 20, 1, 65, 0.16): 0.00021845005647591305, (0, 0, 3, 20, 1, 65, 0.17): 0.00021355442495014514, (0, 0, 3, 20, 1, 65, 0.18): 0.0008603699038568356, (0, 0, 3, 20, 1, 65, 0.19): 0.0016671072035827034, (0, 0, 3, 20, 1, 65, 0.2): 0.0029082006937035036, (0, 0, 3, 20, 1, 70, 0.01): 349.48698117784375, (0, 0, 3, 20, 1, 70, 0.02): 29.686959032488545, (0, 0, 3, 20, 1, 70, 0.03): 6.818869852225085, (0, 0, 3, 20, 1, 70, 0.04): 2.057417783941701, (0, 0, 3, 20, 1, 70, 0.05): 0.8188390219878748, (0, 0, 3, 20, 1, 70, 0.06): 0.38072294166633985, (0, 0, 3, 20, 1, 70, 0.07): 0.17086624779879603, (0, 0, 3, 20, 1, 70, 0.08): 0.09519447241267744, (0, 0, 3, 20, 1, 70, 0.09): 0.04721796753268035, (0, 0, 3, 20, 1, 70, 0.1): 0.026358688187954834, (0, 0, 3, 20, 1, 70, 0.11): 0.012455254129544227, (0, 0, 3, 20, 1, 70, 0.12): 0.00594043757151455, (0, 0, 3, 20, 1, 70, 0.13): 0.0025465388609371194, (0, 0, 3, 20, 1, 70, 0.14): 0.0009290378457356065, (0, 0, 3, 20, 1, 70, 0.15): 0.00013973836215822166, (0, 0, 3, 20, 1, 70, 0.16): 0.0003914628218645894, (0, 0, 3, 20, 1, 70, 0.17): 0.0010791968317767453, (0, 0, 3, 20, 1, 70, 0.18): 0.0021569212963947672, (0, 0, 3, 20, 1, 70, 0.19): 0.0035499757690346354, (0, 0, 3, 20, 1, 70, 0.2): 0.005052058187767349, (0, 0, 3, 20, 1, 75, 0.01): 249.69785468762993, (0, 0, 3, 20, 1, 75, 0.02): 25.3114444905148, (0, 0, 3, 20, 1, 75, 0.03): 5.1554767308645575, (0, 0, 3, 20, 1, 75, 0.04): 1.4777297124031084, (0, 0, 3, 20, 1, 75, 0.05): 0.6381310663288978, (0, 0, 3, 20, 1, 75, 0.06): 0.26545580733628527, (0, 0, 3, 20, 1, 75, 0.07): 0.1304825078123275, (0, 0, 3, 20, 1, 75, 0.08): 0.07014326532985825, (0, 0, 3, 20, 1, 75, 0.09): 0.03341654458561046, (0, 0, 3, 20, 1, 75, 0.1): 0.016164349233214772, (0, 0, 3, 20, 1, 75, 0.11): 0.007485602338124173, (0, 0, 3, 20, 1, 75, 0.12): 0.003267371629903224, (0, 0, 3, 20, 1, 75, 0.13): 0.0010541989865181236, (0, 0, 3, 20, 1, 75, 0.14): 0.00014148449209246662, (0, 0, 3, 20, 1, 75, 0.15): 0.00037006854258989574, (0, 0, 3, 20, 1, 75, 0.16): 0.0012033588952708298, (0, 0, 3, 20, 1, 75, 0.17): 0.0024579403675961324, (0, 0, 3, 20, 1, 75, 0.18): 0.003946962062158821, (0, 0, 3, 20, 1, 75, 0.19): 0.0055536445714876835, (0, 0, 3, 20, 1, 75, 0.2): 0.007391917223227529}

In [ ]:
two_params_sorted(5, 6, mse10)
two_params_sorted(5, 6, mse11)
two_params_sorted(5, 6, mse12)

RMAX for MSE 10-12

MSE 10 - sphere radius 1

3 - (75, 0.04, 0.1495072841271193)
2.75 - (55, 0.05, 0.16230285108614045)
3.15 - (35, 0.09, 0.18358451344832497)
3 - (60, 0.05, 0.18404598888352552)
3.15 - (45, 0.07, 0.1979441823213399)
3.3 - (55, 0.06, 0.20366842762519113)
3.25 - (65, 0.05, 0.20614644370379973)
3 - (50, 0.06, 0.21214732446721105)
2.8 - (70, 0.04, 0.21345673479200758)
2.8 - (35, 0.08, 0.21717343120981802)

MSE 11 - sphere radius 2

6 - (75, 0.08, 0.0038662466058983704)
6.5 - (65, 0.1, 0.003888774346853614)
6.3 - (70, 0.09, 0.0051314553921075435)
6 - (60, 0.1, 0.005988867061248519)
6.5 - (50, 0.13, 0.006344012109814914)
6.05 - (55, 0.11, 0.00636260578005052)
5.85 - (65, 0.09, 0.006367673645517341)
7 - (50, 0.14, 0.006746423331903903)
6.4 - (40, 0.16, 0.006935998657993217)
6.3 - (45, 0.14, 0.00694940885043103)

MSE 12 - sphere radius 3

10.5 - (70, 0.15, 0.00013973836215822166)
10.5 - (75, 0.14, 0.00014148449209246662)
10.8 - (60, 0.18, 0.00016245005782657176)
11.07 - (65, 0.17, 0.00021355442495014514)
10.4 - (65, 0.16, 0.00021845005647591305)
11 - (55, 0.2, 0.00025387499205618814)
10.45 - (55, 0.19, 0.00026164350602219766)
11.25 - (75, 0.15, 0.00037006854258989574)
11.2 - (70, 0.16, 0.0003914628218645894)
10.2 - (60, 0.17, 0.0004296264530680025)

In [ ]:
import math
pi = math.pi
mse13 = populate_specific(2000, [0], [0], [3], [20], [1], None, 1, pi)
mse14 = populate_specific(2000, [0], [0], [3], [20], [1], None, 2, 2*pi)
mse15 = populate_specific(2000, [0], [0], [3], [20], [1], None, 3, 3*pi)

In [ ]:
print(mse13)
print(mse14)
print(mse15)

In [ ]:
hmm = []
for i in range(25):
    hmm.append(populate_specific(2000, [0], [0], [3], [20], [1], None, 1, 3 + 0.01*i))

In [ ]:
new = []
for i in range(25):
    for val in hmm[i].values():
        new.append(val)
print(new)
    

In [ ]:
print(hmm)


In [ ]:
def comp_rs(N, manifolds, distTypes, kfuncs, nns, fits, nballvals, radius, rmax):
    
    diffs = {}

    for manifold in manifolds:
        for distType in distTypes:
            for kfunc in kfuncs:
                for nn in nns:
                    for fitVersion in fits:
                        mser = run_estimator(N, manifold, distType, kfunc, nn, fitVersion, None, None, radius, rmax)
                        for nballs in nballvals:
                            spacing = rmax / nballs
                            mse = run_estimator(N, manifold, distType, kfunc, nn, fitVersion, nballs, spacing, radius, None)
                            diffs[(manifold, distType, kfunc, nn, fitVersion, nballs, spacing)] = mse - mser
                            print('finished', nballs, 'geodesic balls')
    return diffs

Generic binary searches for things

In [ ]:
def bestrNU(N, manifold, distType, kfunc, nn, fit, radius, low, high):
    mses = {}
    currmin = run_estimator(N, manifold, distType, kfunc, nn, fit, 1, None, radius, None)
    if high >= low:
        mid = (high + low) // 2
        mse = run_estimator(N, manifold, distType, kfunc, nn, fit, mid, None, radius, None)
        mseleft = run_estimator(N, manifold, distType, kfunc, nn, fit, mid-1, None, radius, None)
        mseright = run_estimator(N, manifold, distType, kfunc, nn, fit, mid+1, None, radius, None)
        mses[mid] = mse
        if (mseleft > mse) and (mseright > mse):
            return (mses, mid, mse)
        elif mse > mseleft:
            return bestrNU(N, manifold, distType, kfunc, nn, fit, radius, low, mid-1)
        else:
            return bestrNU(N, manifold, distType, kfunc, nn, fit, radius, mid+1, high)
 
    else:
        return (mses, low, "sus")

def bestspacing(N, manifold, distType, kfunc, nn, fit, nballs, radius, low, high, interval):
    mses = {}
    currmin = run_estimator(N, manifold, distType, kfunc, nn, fit, nballs, None, radius, None)
    if high >= low:
        mid = (high + low) // 2
        mse = run_estimator(N, manifold, distType, kfunc, nn, fit, nballs, mid*interval, radius, None)
        mseleft = run_estimator(N, manifold, distType, kfunc, nn, fit, nballs, (mid-1)*interval, radius, None)
        mseright = run_estimator(N, manifold, distType, kfunc, nn, fit, nballs, (mid+1)*interval, radius, None)
        mses[mid] = mse
        if (mseleft > mse) and (mseright > mse):
            return (mses, mid*interval, mse)
        elif mse > mseleft:
            return bestspacing(N, manifold, distType, kfunc, nn, fit, nballs, radius, low, mid-1, interval)
        else:
            return bestspacing(N, manifold, distType, kfunc, nn, fit, nballs, radius, mid+1, high, interval)
 
    else:
        return (mses, low, "sus")

# Note: bestgb is identical to bestrNU except interval, which for rNU is None, so combine into one function maybe?
    
def bestgb(N, manifold, distType, kfunc, nn, fit, radius, low, high, interval):
    mses = {}
    currmin = run_estimator(N, manifold, distType, kfunc, nn, fit, 1, interval, radius, None)
    if high >= low:
        mid = (high + low) // 2
        mse = run_estimator(N, manifold, distType, kfunc, nn, fit, mid, interval, radius, None)
        mseleft = run_estimator(N, manifold, distType, kfunc, nn, fit, mid-1, interval, radius, None)
        mseright = run_estimator(N, manifold, distType, kfunc, nn, fit, mid+1, interval, radius, None)
        mses[mid] = mse
        if (mseleft > mse) and (mseright > mse):
            return (mses, mid, mse)
        elif mse > mseleft:
            return bestgb(N, manifold, distType, kfunc, nn, fit, radius, low, mid-1, interval)
        else:
            return bestgb(N, manifold, distType, kfunc, nn, fit, radius, mid+1, high, interval)
 
    else:
        return (mses, low, "sus")


In [ ]:
def rmax_array(Ns, radii, nballvals, manifold, distType, kfunc, nn, fit, low, high, interval):
    rs = {}
    for N in Ns:
        for radius in radii:
            for nball in nballvals:
                sce = make_sce(N, manifold, radius)
                rs[(N, radius, nball)] = bestspacing(sce, nball, low, high, interval)[1:]
    return rs

RUN CELL BELOW TO SEE RMAX STATS FOR EXP21

In [ ]:
rs = {(1000, 1, 50): (0.059000000000000004, 0.16289564653848623), (1000, 1, 100): (0.029, 0.116031107737186), (1000, 1, 150): (0.018000000000000002, 0.12546403915594795), (1000, 1, 200): (0.013000000000000001, 0.11556913025619218), (1000, 1, 250): (0.009000000000000001, 0.04817953979729957), (1000, 2, 50): (0.047, 0.2500244192306883), (1000, 2, 100): (0.023, 0.1204842623157765), (1000, 2, 150): (0.015, 0.10774332825715038), (1000, 2, 200): (0.011, 0.0635914677199602), (1000, 2, 250): (0.009000000000000001, 0.06420240207056714), (1000, 3, 50): (0.061, 0.031201925154424784), (1000, 3, 100): (0.03, 0.023274155095081312), (1000, 3, 150): (0.02, 0.014773030189863433), (1000, 3, 200): (0.015, 0.021985751649669198), (1000, 3, 250): (0.012, 0.021382847152562258), (2000, 1, 50): (0.062, 0.20325959920592143), (2000, 1, 100): (0.028, 0.14385172432903134), (2000, 1, 150): (0.017, 0.10492326321996287), (2000, 1, 200): (0.012, 0.049778076740717235), (2000, 1, 250): (0.011, 0.10643819077520772), (2000, 2, 50): (0.043000000000000003, 0.6670499349521133), (2000, 2, 100): (0.018000000000000002, 0.4993999798330912), (2000, 2, 150): (0.012, 0.2567348951056555), (2000, 2, 200): (0.009000000000000001, 0.18724963066827063), (2000, 2, 250): (0.007, 0.16797587235542077), (2000, 3, 50): (0.05, 0.132046942055803), (2000, 3, 100): (0.025, 0.05177292620846586), (2000, 3, 150): (0.016, 0.045069247452657946), (2000, 3, 200): (0.012, 0.04901017547402262), (2000, 3, 250): (0.01, 0.031043068577128376), (3000, 1, 50): (0.06, 0.17955577242581486), (3000, 1, 100): (0.029, 0.1636810149602326), (3000, 1, 150): (0.018000000000000002, 0.13263788155717768), (3000, 1, 200): (0.014, 0.13220075970721326), (3000, 1, 250): (0.01, 0.0671708628870581), (3000, 2, 50): (0.044, 1.2765995314243), (3000, 2, 100): (0.017, 0.9908509167236701), (3000, 2, 150): (0.01, 0.5088087160711526), (3000, 2, 200): (0.008, 0.40594115830237343), (3000, 2, 250): (0.006, 0.3566029870301843), (3000, 3, 50): (0.047, 0.33159974466088127), (3000, 3, 100): (0.023, 0.15273604818411082), (3000, 3, 150): (0.015, 0.11308047377000431), (3000, 3, 200): (0.011, 0.0814367410515269), (3000, 3, 250): (0.009000000000000001, 0.050005032380200834), (4000, 1, 50): (0.061, 0.18728385762236024), (4000, 1, 100): (0.028, 0.15314714179008604), (4000, 1, 150): (0.018000000000000002, 0.1457050523051779), (4000, 1, 200): (0.013000000000000001, 0.10239900712494168), (4000, 1, 250): (0.011, 0.13261541448962091), (4000, 2, 50): (0.045, 1.3830130878052405), (4000, 2, 100): (0.019, 1.287805874685234), (4000, 2, 150): (0.01, 0.9182716067764782), (4000, 2, 200): (0.007, 0.8509948615220071), (4000, 2, 250): (0.006, 0.43418219138733805), (4000, 3, 50): (0.045, 0.5071016434747643), (4000, 3, 100): (0.021, 0.2648949943768306), (4000, 3, 150): (0.014, 0.17846055005866346), (4000, 3, 200): (0.01, 0.10433779229853231), (4000, 3, 250): (0.008, 0.09045268355691508), (5000, 1, 50): (0.061, 0.1940194137448843), (5000, 1, 100): (0.028, 0.15850438573332218), (5000, 1, 150): (0.018000000000000002, 0.13142325800716048), (5000, 1, 200): (0.013000000000000001, 0.10844044580816967), (5000, 1, 250): (0.01, 0.07849490386452651), (5000, 2, 50): (0.054, 1.814726947649878), (5000, 2, 100): (0.02, 1.561188293583791), (5000, 2, 150): (0.012, 1.32761597118213), (5000, 2, 200): (0.007, 0.7984253367347969), (5000, 2, 250): (0.005, 0.6997857946217262), (5000, 3, 50): (0.042, 0.7476981806865091), (5000, 3, 100): (0.019, 0.4765717339524199), (5000, 3, 150): (0.013000000000000001, 0.36333844066921184), (5000, 3, 200): (0.009000000000000001, 0.15422439768426793), (5000, 3, 250): (0.007, 0.18923297068521738), (6000, 1, 50): (0.06, 0.17820912681464968), (6000, 1, 100): (0.028, 0.1612456028541741), (6000, 1, 150): (0.018000000000000002, 0.12928415836631074), (6000, 1, 200): (0.013000000000000001, 0.12376662647936704), (6000, 1, 250): (0.01, 0.11439122509735171), (6000, 2, 50): (0.052000000000000005, 1.8251051204100293), (6000, 2, 100): (0.021, 1.5573722245991066), (6000, 2, 150): (0.011, 1.2304204140164805), (6000, 2, 200): (0.007, 0.9202323113063321), (6000, 2, 250): (0.006, 0.9003018046787294), (6000, 3, 50): (0.043000000000000003, 1.1427913167984913), (6000, 3, 100): (0.019, 0.6445019414629963), (6000, 3, 150): (0.012, 0.47700931954807896), (6000, 3, 200): (0.009000000000000001, 0.31441750056903756), (6000, 3, 250): (0.007, 0.22030602850799763), (7000, 1, 50): (0.06, 0.18336247486785207), (7000, 1, 100): (0.029, 0.162734066198087), (7000, 1, 150): (0.018000000000000002, 0.11767336407181303), (7000, 1, 200): (0.013000000000000001, 0.1264931233197759), (7000, 1, 250): (0.01, 0.11042887270887584), (7000, 2, 50): (0.053, 1.8677612707611722), (7000, 2, 100): (0.019, 1.4770132593152303), (7000, 2, 150): (0.011, 1.202266495123628), (7000, 2, 200): (0.008, 1.425067749439478), (7000, 2, 250): (0.005, 1.175615319764963), (7000, 3, 50): (0.042, 1.2732846533662243), (7000, 3, 100): (0.018000000000000002, 0.6899088452899991), (7000, 3, 150): (0.011, 0.5113914840136203), (7000, 3, 200): (0.008, 0.34508452715884663), (7000, 3, 250): (0.006, 0.31178576853215295), (8000, 1, 50): (0.059000000000000004, 0.16999729801325333), (8000, 1, 100): (0.028, 0.1592678105564874), (8000, 1, 150): (0.018000000000000002, 0.12343324394709107), (8000, 1, 200): (0.013000000000000001, 0.11575963079952252), (8000, 1, 250): (0.01, 0.1045508169950162), (8000, 2, 50): (0.054, 1.9285737825979936), (8000, 2, 100): (0.021, 1.6914837123053243), (8000, 2, 150): (0.012, 1.486105405131173), (8000, 2, 200): (0.007, 1.0579879680615822), (8000, 2, 250): (0.005, 0.9059219935007229), (8000, 3, 50): (0.043000000000000003, 1.6409007473718293), (8000, 3, 100): (0.018000000000000002, 1.1470257821677645), (8000, 3, 150): (0.011, 0.691090910565122), (8000, 3, 200): (0.008, 0.5005758659928479), (8000, 3, 250): (0.006, 0.3970871917499336), (9000, 1, 50): (0.06, 0.17872218141888038), (9000, 1, 100): (0.028, 0.14935500385947614), (9000, 1, 150): (0.018000000000000002, 0.13693930611781252), (9000, 1, 200): (0.013000000000000001, 0.13607426936444206), (9000, 1, 250): (0.01, 0.08777122876721244), (9000, 2, 50): (0.054, 1.9632726994868244), (9000, 2, 100): (0.021, 1.7607892743895293), (9000, 2, 150): (0.012, 1.517398642564235), (9000, 2, 200): (0.008, 1.40860922781762), (9000, 2, 250): (0.006, 1.3470750630078523), (9000, 3, 50): (0.043000000000000003, 1.6289564544551616), (9000, 3, 100): (0.018000000000000002, 1.093443481606912), (9000, 3, 150): (0.011, 0.7781014938508147), (9000, 3, 200): (0.008, 0.5468566290795104), (9000, 3, 250): (0.006, 0.45619900677384856), (10000, 1, 50): (0.059000000000000004, 0.1795289010780114), (10000, 1, 100): (0.028, 0.15760228390703343), (10000, 1, 150): (0.018000000000000002, 0.1375150002740513), (10000, 1, 200): (0.013000000000000001, 0.1277782078069474), (10000, 1, 250): (0.01, 0.09453854862948098), (10000, 2, 50): (0.055, 1.9754398512365632), (10000, 2, 100): (0.022, 1.8485999371918636), (10000, 2, 150): (0.013000000000000001, 1.6623741209598393), (10000, 2, 200): (0.008, 1.4825366983077555), (10000, 2, 250): (0.006, 1.2571674255835856), (10000, 3, 50): (0.044, 1.763811993422178), (10000, 3, 100): (0.018000000000000002, 1.3845962284881228), (10000, 3, 150): (0.011, 1.0580810368589522), (10000, 3, 200): (0.008, 0.6840006004962239), (10000, 3, 250): (0.006, 0.45727176275427844)}

# to get rmax, need to multiply nballs by spacing (which is first entry in key)

def getrmaxs(rs):
    rmaxs = {}
    for key in rs.keys():
        spacing, error = rs[key]
        rmaxs[key] = np.array([key[2]*spacing, error, 1])
    return rmaxs
# #print(rmaxs)

def getavgrmax(N, radius, nball, rmaxs):
    vals = np.zeros(3)
    for key in rmaxs.keys():
        if (N == None or N == key[0]) and (radius == None or radius == key[1]) and (nball == None or nball == key[2]):
            vals = vals + rmaxs[key]
    avgr = vals[0]/vals[2]
    avge = vals[1]/vals[2]
    return (avgr, avge)

# should output 2.95, 0.16289565
# print(getavgrmax(1000, 1, 50, rmaxs))

# should output 2.7833333333333337, 0.14804066364119975 poggers
# print(getavgrmax(1000, None, 50, rmaxs))

def getallavgrs(Ns, radii, nballvals, rmaxs):
    allrs = {}
    print('rmax by Ns')
    for N in Ns:
        things = getavgrmax(N, None, None, rmaxs)
        allrs[(N, None, None)] = things
        print(N, things)
    print('rmax by radii')
    for radius in radii:
        things = getavgrmax(None, radius, None, rmaxs)
        allrs[(None, radius, None)] = things
        print(radius, things)
    print('rmax by nballs')
    for nball in nballvals:
        things = getavgrmax(None, None, nball, rmaxs)
        allrs[(None, None, nball)] = things
        print(nball, things)
    print('rmax by N AND radii')
    for N in Ns:
        for radius in radii:
            things = getavgrmax(N, radius, None, rmaxs)
            allrs[(N, radius, None)] = things
            print(N, radius, things)
    print('rmax by N AND nballs')
    for N in Ns:
        for nball in nballvals:
            things = getavgrmax(N, None, nball, rmaxs)
            allrs[(N, None, nball)] = things
            print(N, nball, things)
    print('rmax by radii AND nballs')
    for radius in radii:
        for nball in nballvals:
            things = getavgrmax(None, radius, nball, rmaxs)
            allrs[(None, radius, nball)] = things
            print(radius, nball, things)
    return allrs
    
allrs = getallavgrs([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000], [1, 2, 3], [50, 100, 150, 200, 250], rmaxs)

In [ ]:
for i in range(3):
    print(allrs[(None, i+1, 250)][0])